In [16]:
import os

def find_files_with_pattern(root_path, extensions=['.pcm', '.txt'], max_files=50):
    found_files = []
    
    for root, dirs, files in os.walk(root_path):
        for file in files:
            if any(file.endswith(ext) for ext in extensions):
                full_path = os.path.join(root, file)
                # 상대 경로로 표시
                rel_path = os.path.relpath(full_path, root_path)
                found_files.append(rel_path)
                
                if len(found_files) >= max_files:
                    return found_files
    
    return found_files

# PCM과 TXT 파일 찾기
print("PCM 및 TXT 파일 구조:")
files = find_files_with_pattern(".", ['.pcm', '.txt'], max_files=100)
for file in files:
    print(file)

PCM 및 TXT 파일 구조:
KsponSpeech_0450/KsponSpeech_449656.pcm
KsponSpeech_0450/KsponSpeech_449581.pcm
KsponSpeech_0450/KsponSpeech_449371.pcm
KsponSpeech_0450/KsponSpeech_449256.pcm
KsponSpeech_0450/KsponSpeech_449309.pcm
KsponSpeech_0450/KsponSpeech_449379.pcm
KsponSpeech_0450/KsponSpeech_449333.txt
KsponSpeech_0450/KsponSpeech_449584.txt
KsponSpeech_0450/KsponSpeech_449610.txt
KsponSpeech_0450/KsponSpeech_449085.txt
KsponSpeech_0450/KsponSpeech_449322.pcm
KsponSpeech_0450/KsponSpeech_449530.pcm
KsponSpeech_0450/KsponSpeech_449625.pcm
KsponSpeech_0450/KsponSpeech_449031.txt
KsponSpeech_0450/KsponSpeech_449514.pcm
KsponSpeech_0450/KsponSpeech_449454.pcm
KsponSpeech_0450/KsponSpeech_449733.txt
KsponSpeech_0450/KsponSpeech_449800.txt
KsponSpeech_0450/KsponSpeech_449219.pcm
KsponSpeech_0450/KsponSpeech_449532.pcm
KsponSpeech_0450/KsponSpeech_449102.pcm
KsponSpeech_0450/KsponSpeech_449248.txt
KsponSpeech_0450/KsponSpeech_449700.txt
KsponSpeech_0450/KsponSpeech_449175.txt
KsponSpeech_0450/KsponS

In [3]:
cd KsponSpeech_04

/data/Traindata_2/KsponSpeech_04


In [ ]:
ls

In [17]:
import os
import glob
import numpy as np
import librosa
from g2pk import G2p
from tqdm import tqdm
import struct
from pathlib import Path

# G2p 초기화
g2p = G2p()

# 기본 디렉토리 경로 설정
base_dir = "."  # 현재 디렉토리 (KsponSpeech_0450이 있는 곳)
target_base_dir = "./PreprocessData_04"  # 전처리된 데이터를 저장할 기본 디렉토리

# PCM 파일을 읽고 오디오 데이터로 변환하는 함수
def read_pcm_file(pcm_file, sample_rate=16000, channels=1, bit_depth=16):
    """
    PCM 파일을 읽어서 numpy 배열로 변환합니다.
    
    Args:
        pcm_file: PCM 파일 경로
        sample_rate: 샘플링 레이트 (기본값: 16000)
        channels: 채널 수 (기본값: 1, 모노)
        bit_depth: 비트 깊이 (기본값: 16)
    
    Returns:
        numpy array: 오디오 데이터
    """
    try:
        with open(pcm_file, 'rb') as f:
            # PCM 데이터 읽기
            raw_data = f.read()
        
        # 16비트 PCM 데이터를 numpy 배열로 변환
        if bit_depth == 16:
            audio_data = np.frombuffer(raw_data, dtype=np.int16)
        elif bit_depth == 32:
            audio_data = np.frombuffer(raw_data, dtype=np.int32)
        else:
            raise ValueError(f"지원하지 않는 비트 깊이: {bit_depth}")
        
        # 정규화 (-1.0 ~ 1.0 범위로)
        if bit_depth == 16:
            audio_data = audio_data.astype(np.float32) / 32768.0
        elif bit_depth == 32:
            audio_data = audio_data.astype(np.float32) / 2147483648.0
        
        return audio_data
    
    except Exception as e:
        print(f"PCM 파일 읽기 오류 ({pcm_file}): {str(e)}")
        return None

# 오디오 특성 추출 함수
def extract_audio_features(audio_data, sample_rate=16000):
    """
    오디오 데이터에서 다양한 특성을 추출합니다.
    
    Args:
        audio_data: 오디오 데이터 (numpy array)
        sample_rate: 샘플링 레이트
    
    Returns:
        dict: 추출된 특성들
    """
    try:
        # 1. MFCC 특성 추출
        mfcc = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=13)
        
        # 2. 멜 스펙트로그램
        mel_spec = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate, n_mels=80)
        log_mel_spec = librosa.power_to_db(mel_spec)
        
        # 3. 스펙트럴 센트로이드 (음색의 밝기)
        spectral_centroids = librosa.feature.spectral_centroid(y=audio_data, sr=sample_rate)
        
        # 4. 스펙트럴 롤오프
        spectral_rolloff = librosa.feature.spectral_rolloff(y=audio_data, sr=sample_rate)
        
        # 5. 제로 크로싱 레이트
        zcr = librosa.feature.zero_crossing_rate(audio_data)
        
        # 6. 크로마 특성
        chroma = librosa.feature.chroma_stft(y=audio_data, sr=sample_rate)
        
        # 7. 피치(F0) 추출
        f0, voiced_flag, voiced_probs = librosa.pyin(audio_data, 
                                                   fmin=librosa.note_to_hz('C2'), 
                                                   fmax=librosa.note_to_hz('C7'))
        
        features = {
            'mfcc': mfcc,
            'mel_spectrogram': log_mel_spec,
            'spectral_centroids': spectral_centroids,
            'spectral_rolloff': spectral_rolloff,
            'zero_crossing_rate': zcr,
            'chroma': chroma,
            'f0': f0,
            'voiced_flag': voiced_flag,
            'voiced_probs': voiced_probs,
            'sample_rate': sample_rate,
            'duration': len(audio_data) / sample_rate
        }
        
        return features
    
    except Exception as e:
        print(f"특성 추출 오류: {str(e)}")
        return None

# 텍스트 파일 로드 함수 (다양한 인코딩 시도)
def load_text(text_file):
    """다양한 인코딩을 시도하여 텍스트 파일을 로드합니다."""
    encodings = ['utf-8', 'cp949', 'euc-kr', 'latin1']
    
    for encoding in encodings:
        try:
            with open(text_file, 'r', encoding=encoding) as f:
                return f.read().strip()
        except UnicodeDecodeError:
            continue
    
    # 모든 인코딩이 실패하면 바이너리 모드로 읽고 강제 디코딩
    try:
        with open(text_file, 'rb') as f:
            raw_data = f.read()
            return raw_data.decode('latin1').encode('utf-8').decode('utf-8')
    except Exception as e:
        raise ValueError(f"모든 인코딩 시도 실패: {str(e)}")

# 디렉토리 내의 PCM 및 TXT 파일 경로 가져오기
def get_file_paths(directory):
    """디렉토리 내의 모든 PCM 및 TXT 파일 경로를 반환합니다."""
    pcm_files = sorted(glob.glob(os.path.join(directory, "*.pcm")))
    txt_files = sorted(glob.glob(os.path.join(directory, "*.txt")))
    return pcm_files, txt_files

# 메인 전처리 함수
def preprocess_kspon_data():
    """KsponSpeech_04 데이터를 전처리합니다."""
    
    # KsponSpeech_0450 디렉토리 처리
    source_dir = "./KsponSpeech_0450"
    
    # 저장할 디렉토리들 생성
    audio_target_dir = f"{target_base_dir}/audio_features"  # NPZ 파일들
    text_target_dir = f"{target_base_dir}/g2p_texts"       # G2P 변환된 텍스트들
    
    print(f"처리할 디렉토리: {source_dir}")
    print(f"오디오 특성 저장 디렉토리: {audio_target_dir}")
    print(f"G2P 텍스트 저장 디렉토리: {text_target_dir}")
    
    # 소스 디렉토리가 존재하는지 확인
    if not os.path.exists(source_dir):
        print(f"소스 디렉토리가 존재하지 않습니다: {source_dir}")
        return
    
    # 대상 디렉토리들 생성
    os.makedirs(audio_target_dir, exist_ok=True)
    os.makedirs(text_target_dir, exist_ok=True)
    
    # 파일 경로 가져오기
    pcm_files, txt_files = get_file_paths(source_dir)
    
    print(f"발견된 PCM 파일: {len(pcm_files)}개")
    print(f"발견된 TXT 파일: {len(txt_files)}개")
    
    # 통계 변수
    audio_success = 0
    audio_skip = 0
    audio_error = 0
    
    text_success = 0
    text_skip = 0
    text_error = 0
    
    print("\n=== PCM 파일 처리 시작 ===")
    
    # PCM 파일들 처리 (오디오 특성 추출)
    for pcm_file in tqdm(pcm_files, desc="PCM 파일 처리"):
        # 파일명 추출 (예: KsponSpeech_449656)
        base_name = os.path.basename(pcm_file).split('.')[0]
        target_file = os.path.join(audio_target_dir, f"{base_name}.npz")
        
        # 이미 처리된 파일인지 확인
        if os.path.exists(target_file):
            audio_skip += 1
            continue
        
        try:
            # PCM 파일 읽기
            audio_data = read_pcm_file(pcm_file)
            
            if audio_data is None:
                audio_error += 1
                continue
            
            # 오디오 특성 추출
            features = extract_audio_features(audio_data)
            
            if features is None:
                audio_error += 1
                continue
            
            # NPZ 파일로 저장
            np.savez_compressed(target_file, **features)
            
            audio_success += 1
            
        except Exception as e:
            audio_error += 1
            print(f"오디오 처리 오류 ({base_name}): {str(e)}")
    
    print(f"\nPCM 파일 처리 결과:")
    print(f"  성공: {audio_success}개")
    print(f"  건너뜀 (이미 처리됨): {audio_skip}개")
    print(f"  오류: {audio_error}개")
    
    print("\n=== TXT 파일 처리 시작 ===")
    
    # TXT 파일들 처리 (G2P 변환)
    for txt_file in tqdm(txt_files, desc="TXT 파일 처리"):
        # 파일명 추출 (예: KsponSpeech_449656)
        base_name = os.path.basename(txt_file).split('.')[0]
        target_file = os.path.join(text_target_dir, f"{base_name}.txt")
        
        # 이미 처리된 파일인지 확인
        if os.path.exists(target_file):
            text_skip += 1
            continue
        
        try:
            # 텍스트 로드
            original_text = load_text(txt_file)
            
            # G2P 변환 적용
            g2p_text = g2p(original_text)
            
            # 결과 저장
            with open(target_file, 'w', encoding='utf-8') as f:
                f.write(g2p_text)
            
            text_success += 1
            
        except Exception as e:
            text_error += 1
            print(f"텍스트 처리 오류 ({base_name}): {str(e)}")
    
    print(f"\nTXT 파일 처리 결과:")
    print(f"  성공: {text_success}개")
    print(f"  건너뜀 (이미 처리됨): {text_skip}개")
    print(f"  오류: {text_error}개")
    
    print(f"\n=== 전체 처리 완료 ===")
    print(f"전처리된 데이터 저장 위치: {target_base_dir}")
    print(f"  - 오디오 특성 (NPZ): {audio_target_dir}")
    print(f"  - G2P 텍스트: {text_target_dir}")
    
    # 매칭 확인
    processed_audio_files = len([f for f in os.listdir(audio_target_dir) if f.endswith('.npz')])
    processed_text_files = len([f for f in os.listdir(text_target_dir) if f.endswith('.txt')])
    
    print(f"\n=== 최종 통계 ===")
    print(f"처리된 오디오 파일: {processed_audio_files}개")
    print(f"처리된 텍스트 파일: {processed_text_files}개")
    
    if processed_audio_files == processed_text_files:
        print("✅ 오디오와 텍스트 파일 수가 일치합니다!")
    else:
        print("⚠️ 오디오와 텍스트 파일 수가 일치하지 않습니다.")

# 실행
if __name__ == "__main__":
    print("KsponSpeech_04 데이터 전처리를 시작합니다...")
    preprocess_kspon_data()

KsponSpeech_04 데이터 전처리를 시작합니다...
처리할 디렉토리: ./KsponSpeech_0450
오디오 특성 저장 디렉토리: ./PreprocessData_04/audio_features
G2P 텍스트 저장 디렉토리: ./PreprocessData_04/g2p_texts
발견된 PCM 파일: 1000개
발견된 TXT 파일: 1000개

=== PCM 파일 처리 시작 ===


PCM 파일 처리: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [18:24<00:00,  1.10s/it]



PCM 파일 처리 결과:
  성공: 1000개
  건너뜀 (이미 처리됨): 0개
  오류: 0개

=== TXT 파일 처리 시작 ===


TXT 파일 처리: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:27<00:00, 36.57it/s]


TXT 파일 처리 결과:
  성공: 1000개
  건너뜀 (이미 처리됨): 0개
  오류: 0개

=== 전체 처리 완료 ===
전처리된 데이터 저장 위치: ./PreprocessData_04
  - 오디오 특성 (NPZ): ./PreprocessData_04/audio_features
  - G2P 텍스트: ./PreprocessData_04/g2p_texts

=== 최종 통계 ===
처리된 오디오 파일: 1000개
처리된 텍스트 파일: 1000개
✅ 오디오와 텍스트 파일 수가 일치합니다!


In [18]:
ls

KsponSpeech_0373/  KsponSpeech_0405/  KsponSpeech_0437/  KsponSpeech_0469/
KsponSpeech_0374/  KsponSpeech_0406/  KsponSpeech_0438/  KsponSpeech_0470/
KsponSpeech_0375/  KsponSpeech_0407/  KsponSpeech_0439/  KsponSpeech_0471/
KsponSpeech_0376/  KsponSpeech_0408/  KsponSpeech_0440/  KsponSpeech_0472/
KsponSpeech_0377/  KsponSpeech_0409/  KsponSpeech_0441/  KsponSpeech_0473/
KsponSpeech_0378/  KsponSpeech_0410/  KsponSpeech_0442/  KsponSpeech_0474/
KsponSpeech_0379/  KsponSpeech_0411/  KsponSpeech_0443/  KsponSpeech_0475/
KsponSpeech_0380/  KsponSpeech_0412/  KsponSpeech_0444/  KsponSpeech_0476/
KsponSpeech_0381/  KsponSpeech_0413/  KsponSpeech_0445/  KsponSpeech_0477/
KsponSpeech_0382/  KsponSpeech_0414/  KsponSpeech_0446/  KsponSpeech_0478/
KsponSpeech_0383/  KsponSpeech_0415/  KsponSpeech_0447/  KsponSpeech_0479/
KsponSpeech_0384/  KsponSpeech_0416/  KsponSpeech_0448/  KsponSpeech_0480/
KsponSpeech_0385/  KsponSpeech_0417/  KsponSpeech_0449/  KsponSpeech_0481/
KsponSpeech_0386/  KsponS

In [19]:
import os
import glob
import numpy as np
import librosa
from g2pk import G2p
from tqdm import tqdm
import struct
from pathlib import Path

# G2p 초기화
g2p = G2p()

# 기본 디렉토리 경로 설정
base_dir = "."  # 현재 디렉토리 (KsponSpeech_0450이 있는 곳)
target_base_dir = "./PreprocessData_04"  # 전처리된 데이터를 저장할 기본 디렉토리

# PCM 파일을 읽고 오디오 데이터로 변환하는 함수
def read_pcm_file(pcm_file, sample_rate=16000, channels=1, bit_depth=16):
    """
    PCM 파일을 읽어서 numpy 배열로 변환합니다.
    
    Args:
        pcm_file: PCM 파일 경로
        sample_rate: 샘플링 레이트 (기본값: 16000)
        channels: 채널 수 (기본값: 1, 모노)
        bit_depth: 비트 깊이 (기본값: 16)
    
    Returns:
        numpy array: 오디오 데이터
    """
    try:
        with open(pcm_file, 'rb') as f:
            # PCM 데이터 읽기
            raw_data = f.read()
        
        # 16비트 PCM 데이터를 numpy 배열로 변환
        if bit_depth == 16:
            audio_data = np.frombuffer(raw_data, dtype=np.int16)
        elif bit_depth == 32:
            audio_data = np.frombuffer(raw_data, dtype=np.int32)
        else:
            raise ValueError(f"지원하지 않는 비트 깊이: {bit_depth}")
        
        # 정규화 (-1.0 ~ 1.0 범위로)
        if bit_depth == 16:
            audio_data = audio_data.astype(np.float32) / 32768.0
        elif bit_depth == 32:
            audio_data = audio_data.astype(np.float32) / 2147483648.0
        
        return audio_data
    
    except Exception as e:
        print(f"PCM 파일 읽기 오류 ({pcm_file}): {str(e)}")
        return None

# 오디오 특성 추출 함수
def extract_audio_features(audio_data, sample_rate=16000):
    """
    오디오 데이터에서 다양한 특성을 추출합니다.
    
    Args:
        audio_data: 오디오 데이터 (numpy array)
        sample_rate: 샘플링 레이트
    
    Returns:
        dict: 추출된 특성들
    """
    try:
        # 1. MFCC 특성 추출
        mfcc = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=13)
        
        # 2. 멜 스펙트로그램
        mel_spec = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate, n_mels=80)
        log_mel_spec = librosa.power_to_db(mel_spec)
        
        # 3. 스펙트럴 센트로이드 (음색의 밝기)
        spectral_centroids = librosa.feature.spectral_centroid(y=audio_data, sr=sample_rate)
        
        # 4. 스펙트럴 롤오프
        spectral_rolloff = librosa.feature.spectral_rolloff(y=audio_data, sr=sample_rate)
        
        # 5. 제로 크로싱 레이트
        zcr = librosa.feature.zero_crossing_rate(audio_data)
        
        # 6. 크로마 특성
        chroma = librosa.feature.chroma_stft(y=audio_data, sr=sample_rate)
        
        # 7. 피치(F0) 추출
        f0, voiced_flag, voiced_probs = librosa.pyin(audio_data, 
                                                   fmin=librosa.note_to_hz('C2'), 
                                                   fmax=librosa.note_to_hz('C7'))
        
        features = {
            'mfcc': mfcc,
            'mel_spectrogram': log_mel_spec,
            'spectral_centroids': spectral_centroids,
            'spectral_rolloff': spectral_rolloff,
            'zero_crossing_rate': zcr,
            'chroma': chroma,
            'f0': f0,
            'voiced_flag': voiced_flag,
            'voiced_probs': voiced_probs,
            'sample_rate': sample_rate,
            'duration': len(audio_data) / sample_rate
        }
        
        return features
    
    except Exception as e:
        print(f"특성 추출 오류: {str(e)}")
        return None

# 텍스트 파일 로드 함수 (다양한 인코딩 시도)
def load_text(text_file):
    """다양한 인코딩을 시도하여 텍스트 파일을 로드합니다."""
    encodings = ['utf-8', 'cp949', 'euc-kr', 'latin1']
    
    for encoding in encodings:
        try:
            with open(text_file, 'r', encoding=encoding) as f:
                return f.read().strip()
        except UnicodeDecodeError:
            continue
    
    # 모든 인코딩이 실패하면 바이너리 모드로 읽고 강제 디코딩
    try:
        with open(text_file, 'rb') as f:
            raw_data = f.read()
            return raw_data.decode('latin1').encode('utf-8').decode('utf-8')
    except Exception as e:
        raise ValueError(f"모든 인코딩 시도 실패: {str(e)}")

# 디렉토리 내의 PCM 및 TXT 파일 경로 가져오기
def get_file_paths(directory):
    """디렉토리 내의 모든 PCM 및 TXT 파일 경로를 반환합니다."""
    pcm_files = sorted(glob.glob(os.path.join(directory, "*.pcm")))
    txt_files = sorted(glob.glob(os.path.join(directory, "*.txt")))
    return pcm_files, txt_files

# 단일 폴더 처리 함수
def process_single_folder(source_dir, audio_target_dir, text_target_dir):
    """단일 폴더를 처리합니다."""
    
    # 파일 경로 가져오기
    pcm_files, txt_files = get_file_paths(source_dir)
    
    # 통계 변수
    audio_success = 0
    audio_skip = 0
    audio_error = 0
    
    text_success = 0
    text_skip = 0
    text_error = 0
    
    # PCM 파일들 처리 (오디오 특성 추출)
    for pcm_file in tqdm(pcm_files, desc=f"PCM 파일 처리 ({os.path.basename(source_dir)})"):
        # 파일명 추출 (예: KsponSpeech_449656)
        base_name = os.path.basename(pcm_file).split('.')[0]
        target_file = os.path.join(audio_target_dir, f"{base_name}.npz")
        
        # 이미 처리된 파일인지 확인
        if os.path.exists(target_file):
            audio_skip += 1
            continue
        
        try:
            # PCM 파일 읽기
            audio_data = read_pcm_file(pcm_file)
            
            if audio_data is None:
                audio_error += 1
                continue
            
            # 오디오 특성 추출
            features = extract_audio_features(audio_data)
            
            if features is None:
                audio_error += 1
                continue
            
            # NPZ 파일로 저장
            np.savez_compressed(target_file, **features)
            
            audio_success += 1
            
        except Exception as e:
            audio_error += 1
            print(f"오디오 처리 오류 ({base_name}): {str(e)}")
    
    # TXT 파일들 처리 (G2P 변환)
    for txt_file in tqdm(txt_files, desc=f"TXT 파일 처리 ({os.path.basename(source_dir)})"):
        # 파일명 추출 (예: KsponSpeech_449656)
        base_name = os.path.basename(txt_file).split('.')[0]
        target_file = os.path.join(text_target_dir, f"{base_name}.txt")
        
        # 이미 처리된 파일인지 확인
        if os.path.exists(target_file):
            text_skip += 1
            continue
        
        try:
            # 텍스트 로드
            original_text = load_text(txt_file)
            
            # G2P 변환 적용
            g2p_text = g2p(original_text)
            
            # 결과 저장
            with open(target_file, 'w', encoding='utf-8') as f:
                f.write(g2p_text)
            
            text_success += 1
            
        except Exception as e:
            text_error += 1
            print(f"텍스트 처리 오류 ({base_name}): {str(e)}")
    
    return {
        'folder': os.path.basename(source_dir),
        'pcm_files': len(pcm_files),
        'txt_files': len(txt_files),
        'audio_success': audio_success,
        'audio_skip': audio_skip,
        'audio_error': audio_error,
        'text_success': text_success,
        'text_skip': text_skip,
        'text_error': text_error
    }

# 메인 전처리 함수
def preprocess_kspon_data():
    """KsponSpeech_04 모든 폴더의 데이터를 전처리합니다."""
    
    # 현재 디렉토리에서 KsponSpeech_로 시작하는 모든 폴더 찾기 (0450 제외)
    all_folders = sorted([d for d in os.listdir('.') if d.startswith('KsponSpeech_') and os.path.isdir(d) and d != 'KsponSpeech_0450'])
    
    print(f"발견된 KsponSpeech 폴더들: {len(all_folders)}개")
    for folder in all_folders:
        print(f"  - {folder}")
    
    # 저장할 디렉토리들 생성
    audio_target_dir = f"{target_base_dir}/audio_features"  # NPZ 파일들
    text_target_dir = f"{target_base_dir}/g2p_texts"       # G2P 변환된 텍스트들
    
    print(f"\n전처리된 데이터 저장 위치: {target_base_dir}")
    print(f"  - 오디오 특성 (NPZ): {audio_target_dir}")
    print(f"  - G2P 텍스트: {text_target_dir}")
    
    # 대상 디렉토리들 생성
    os.makedirs(audio_target_dir, exist_ok=True)
    os.makedirs(text_target_dir, exist_ok=True)
    
    # 전체 통계
    total_results = []
    total_audio_success = 0
    total_audio_skip = 0
    total_audio_error = 0
    total_text_success = 0
    total_text_skip = 0
    total_text_error = 0
    
    print("\n" + "="*80)
    print("KsponSpeech_04 전체 폴더 처리 시작")
    print("="*80)
    
    # 각 폴더 처리
    for i, folder in enumerate(all_folders, 1):
        source_dir = f"./{folder}"
        
        print(f"\n[{i}/{len(all_folders)}] {folder} 처리 중...")
        
        # 소스 디렉토리가 존재하는지 확인
        if not os.path.exists(source_dir):
            print(f"소스 디렉토리가 존재하지 않습니다: {source_dir}, 건너뜁니다.")
            continue
        
        # 단일 폴더 처리
        result = process_single_folder(source_dir, audio_target_dir, text_target_dir)
        total_results.append(result)
        
        # 통계 누적
        total_audio_success += result['audio_success']
        total_audio_skip += result['audio_skip']
        total_audio_error += result['audio_error']
        total_text_success += result['text_success']
        total_text_skip += result['text_skip']
        total_text_error += result['text_error']
        
        print(f"  {folder} 완료:")
        print(f"    파일 수: PCM {result['pcm_files']}개, TXT {result['txt_files']}개")
        print(f"    오디오: 성공 {result['audio_success']}, 건너뜀 {result['audio_skip']}, 오류 {result['audio_error']}")
        print(f"    텍스트: 성공 {result['text_success']}, 건너뜀 {result['text_skip']}, 오류 {result['text_error']}")
    
    print("\n" + "="*80)
    print("전체 처리 완료!")
    print("="*80)
    
    print(f"\n=== 전체 통계 ===")
    print(f"처리된 폴더 수: {len(total_results)}개")
    print(f"오디오 파일:")
    print(f"  성공: {total_audio_success}개")
    print(f"  건너뜀 (이미 처리됨): {total_audio_skip}개")
    print(f"  오류: {total_audio_error}개")
    print(f"텍스트 파일:")
    print(f"  성공: {total_text_success}개")
    print(f"  건너뜀 (이미 처리됨): {total_text_skip}개")
    print(f"  오류: {total_text_error}개")
    
    # 최종 파일 수 확인
    try:
        processed_audio_files = len([f for f in os.listdir(audio_target_dir) if f.endswith('.npz')])
        processed_text_files = len([f for f in os.listdir(text_target_dir) if f.endswith('.txt')])
        
        print(f"\n=== 최종 결과 ===")
        print(f"저장된 오디오 특성 파일: {processed_audio_files}개")
        print(f"저장된 G2P 텍스트 파일: {processed_text_files}개")
        
        if processed_audio_files == processed_text_files:
            print("✅ 오디오와 텍스트 파일 수가 일치합니다!")
        else:
            print("⚠️ 오디오와 텍스트 파일 수가 일치하지 않습니다.")
            
    except Exception as e:
        print(f"최종 파일 수 확인 중 오류: {str(e)}")
    
    return total_results

# 실행
if __name__ == "__main__":
    print("KsponSpeech_04 데이터 전처리를 시작합니다...")
    preprocess_kspon_data()

KsponSpeech_04 데이터 전처리를 시작합니다...
발견된 KsponSpeech 폴더들: 123개
  - KsponSpeech_0373
  - KsponSpeech_0374
  - KsponSpeech_0375
  - KsponSpeech_0376
  - KsponSpeech_0377
  - KsponSpeech_0378
  - KsponSpeech_0379
  - KsponSpeech_0380
  - KsponSpeech_0381
  - KsponSpeech_0382
  - KsponSpeech_0383
  - KsponSpeech_0384
  - KsponSpeech_0385
  - KsponSpeech_0386
  - KsponSpeech_0387
  - KsponSpeech_0388
  - KsponSpeech_0389
  - KsponSpeech_0390
  - KsponSpeech_0391
  - KsponSpeech_0392
  - KsponSpeech_0393
  - KsponSpeech_0394
  - KsponSpeech_0395
  - KsponSpeech_0396
  - KsponSpeech_0397
  - KsponSpeech_0398
  - KsponSpeech_0399
  - KsponSpeech_0400
  - KsponSpeech_0401
  - KsponSpeech_0402
  - KsponSpeech_0403
  - KsponSpeech_0404
  - KsponSpeech_0405
  - KsponSpeech_0406
  - KsponSpeech_0407
  - KsponSpeech_0408
  - KsponSpeech_0409
  - KsponSpeech_0410
  - KsponSpeech_0411
  - KsponSpeech_0412
  - KsponSpeech_0413
  - KsponSpeech_0414
  - KsponSpeech_0415
  - KsponSpeech_0416
  - KsponSpeech_0

TXT 파일 처리 (KsponSpeech_0373): 100%|███████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:27<00:00, 36.69it/s]


  KsponSpeech_0373 완료:
    파일 수: PCM 1000개, TXT 1000개
    오디오: 성공 1000, 건너뜀 0, 오류 0
    텍스트: 성공 1000, 건너뜀 0, 오류 0

[2/123] KsponSpeech_0374 처리 중...


TXT 파일 처리 (KsponSpeech_0374): 100%|███████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:27<00:00, 36.98it/s]


  KsponSpeech_0374 완료:
    파일 수: PCM 1000개, TXT 1000개
    오디오: 성공 1000, 건너뜀 0, 오류 0
    텍스트: 성공 1000, 건너뜀 0, 오류 0

[3/123] KsponSpeech_0375 처리 중...


PCM 파일 처리 (KsponSpeech_0375):  15%|████████████▏                                                                   | 152/1000 [02:29<13:55,  1.01it/s]


KeyboardInterrupt: 

In [ ]:
import os
import glob
import numpy as np
import librosa
from g2pk import G2p
from tqdm import tqdm
import struct
from pathlib import Path
import multiprocessing as mp
from functools import partial
import time

# 기본 디렉토리 경로 설정
base_dir = "."  # 현재 디렉토리 (KsponSpeech_0450이 있는 곳)
target_base_dir = "./PreprocessData_04"  # 전처리된 데이터를 저장할 기본 디렉토리

# PCM 파일을 읽고 오디오 데이터로 변환하는 함수
def read_pcm_file(pcm_file, sample_rate=16000, channels=1, bit_depth=16):
    """
    PCM 파일을 읽어서 numpy 배열로 변환합니다.
    """
    try:
        with open(pcm_file, 'rb') as f:
            raw_data = f.read()
        
        if bit_depth == 16:
            audio_data = np.frombuffer(raw_data, dtype=np.int16)
            audio_data = audio_data.astype(np.float32) / 32768.0
        elif bit_depth == 32:
            audio_data = np.frombuffer(raw_data, dtype=np.int32)
            audio_data = audio_data.astype(np.float32) / 2147483648.0
        else:
            raise ValueError(f"지원하지 않는 비트 깊이: {bit_depth}")
        
        return audio_data
    
    except Exception as e:
        print(f"PCM 파일 읽기 오류 ({pcm_file}): {str(e)}")
        return None

# 오디오 특성 추출 함수
def extract_audio_features(audio_data, sample_rate=16000):
    """
    오디오 데이터에서 다양한 특성을 추출합니다.
    """
    try:
        # 1. MFCC 특성 추출
        mfcc = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=13)
        
        # 2. 멜 스펙트로그램
        mel_spec = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate, n_mels=80)
        log_mel_spec = librosa.power_to_db(mel_spec)
        
        # 3. 스펙트럴 센트로이드 (음색의 밝기)
        spectral_centroids = librosa.feature.spectral_centroid(y=audio_data, sr=sample_rate)
        
        # 4. 스펙트럴 롤오프
        spectral_rolloff = librosa.feature.spectral_rolloff(y=audio_data, sr=sample_rate)
        
        # 5. 제로 크로싱 레이트
        zcr = librosa.feature.zero_crossing_rate(audio_data)
        
        # 6. 크로마 특성
        chroma = librosa.feature.chroma_stft(y=audio_data, sr=sample_rate)
        
        # 7. 피치(F0) 추출
        f0, voiced_flag, voiced_probs = librosa.pyin(audio_data, 
                                                   fmin=librosa.note_to_hz('C2'), 
                                                   fmax=librosa.note_to_hz('C7'))
        
        features = {
            'mfcc': mfcc,
            'mel_spectrogram': log_mel_spec,
            'spectral_centroids': spectral_centroids,
            'spectral_rolloff': spectral_rolloff,
            'zero_crossing_rate': zcr,
            'chroma': chroma,
            'f0': f0,
            'voiced_flag': voiced_flag,
            'voiced_probs': voiced_probs,
            'sample_rate': sample_rate,
            'duration': len(audio_data) / sample_rate
        }
        
        return features
    
    except Exception as e:
        print(f"특성 추출 오류: {str(e)}")
        return None

# 텍스트 파일 로드 함수 (다양한 인코딩 시도)
def load_text(text_file):
    """다양한 인코딩을 시도하여 텍스트 파일을 로드합니다."""
    encodings = ['utf-8', 'cp949', 'euc-kr', 'latin1']
    
    for encoding in encodings:
        try:
            with open(text_file, 'r', encoding=encoding) as f:
                return f.read().strip()
        except UnicodeDecodeError:
            continue
    
    try:
        with open(text_file, 'rb') as f:
            raw_data = f.read()
            return raw_data.decode('latin1').encode('utf-8').decode('utf-8')
    except Exception as e:
        raise ValueError(f"모든 인코딩 시도 실패: {str(e)}")

# 단일 PCM 파일 처리 함수 (멀티프로세싱용)
def process_single_audio_file(args):
    """단일 PCM 파일을 처리합니다 (멀티프로세싱용)"""
    pcm_file, audio_target_dir = args
    
    base_name = os.path.basename(pcm_file).split('.')[0]
    target_file = os.path.join(audio_target_dir, f"{base_name}.npz")
    
    # 이미 처리된 파일인지 확인
    if os.path.exists(target_file):
        return {'status': 'skip', 'file': base_name}
    
    try:
        # PCM 파일 읽기
        audio_data = read_pcm_file(pcm_file)
        
        if audio_data is None:
            return {'status': 'error', 'file': base_name, 'error': 'PCM 읽기 실패'}
        
        # 오디오 특성 추출
        features = extract_audio_features(audio_data)
        
        if features is None:
            return {'status': 'error', 'file': base_name, 'error': '특성 추출 실패'}
        
        # NPZ 파일로 저장
        np.savez_compressed(target_file, **features)
        
        return {'status': 'success', 'file': base_name}
        
    except Exception as e:
        return {'status': 'error', 'file': base_name, 'error': str(e)}

# 단일 TXT 파일 처리 함수 (멀티프로세싱용)
def process_single_text_file(args):
    """단일 TXT 파일을 처리합니다 (멀티프로세싱용)"""
    txt_file, text_target_dir = args
    
    base_name = os.path.basename(txt_file).split('.')[0]
    target_file = os.path.join(text_target_dir, f"{base_name}.txt")
    
    # 이미 처리된 파일인지 확인
    if os.path.exists(target_file):
        return {'status': 'skip', 'file': base_name}
    
    try:
        # G2p 초기화 (각 프로세스마다)
        g2p = G2p()
        
        # 텍스트 로드
        original_text = load_text(txt_file)
        
        # G2P 변환 적용
        g2p_text = g2p(original_text)
        
        # 결과 저장
        with open(target_file, 'w', encoding='utf-8') as f:
            f.write(g2p_text)
        
        return {'status': 'success', 'file': base_name}
        
    except Exception as e:
        return {'status': 'error', 'file': base_name, 'error': str(e)}

# 디렉토리 내의 PCM 및 TXT 파일 경로 가져오기
def get_file_paths(directory):
    """디렉토리 내의 모든 PCM 및 TXT 파일 경로를 반환합니다."""
    pcm_files = sorted(glob.glob(os.path.join(directory, "*.pcm")))
    txt_files = sorted(glob.glob(os.path.join(directory, "*.txt")))
    return pcm_files, txt_files

# 단일 폴더 처리 함수 (멀티프로세싱 적용)
def process_single_folder_mp(source_dir, audio_target_dir, text_target_dir, num_processes=None):
    """단일 폴더를 멀티프로세싱으로 처리합니다."""
    
    # CPU 코어 수 자동 설정
    if num_processes is None:
        num_processes = min(mp.cpu_count(), 8)  # 최대 8개 프로세스
    
    print(f"🔄 멀티프로세싱 시작: {num_processes}개 프로세스 사용")
    
    # 파일 경로 가져오기
    pcm_files, txt_files = get_file_paths(source_dir)
    
    folder_name = os.path.basename(source_dir)
    
    # 통계 변수
    audio_success = audio_skip = audio_error = 0
    text_success = text_skip = text_error = 0
    
    # 1. PCM 파일들 병렬 처리
    if pcm_files:
        print(f"  🎵 PCM 파일 {len(pcm_files)}개 병렬 처리 중...")
        
        audio_args = [(pcm_file, audio_target_dir) for pcm_file in pcm_files]
        
        with mp.Pool(processes=num_processes) as pool:
            # 진행률 표시를 위한 imap 사용
            results = []
            with tqdm(total=len(audio_args), desc=f"PCM 처리 ({folder_name})") as pbar:
                for result in pool.imap(process_single_audio_file, audio_args):
                    results.append(result)
                    pbar.update(1)
        
        # 결과 집계
        for result in results:
            if result['status'] == 'success':
                audio_success += 1
            elif result['status'] == 'skip':
                audio_skip += 1
            else:
                audio_error += 1
                print(f"    ❌ 오디오 오류 ({result['file']}): {result.get('error', '알 수 없음')}")
    
    # 2. TXT 파일들 병렬 처리
    if txt_files:
        print(f"  📝 TXT 파일 {len(txt_files)}개 병렬 처리 중...")
        
        text_args = [(txt_file, text_target_dir) for txt_file in txt_files]
        
        with mp.Pool(processes=num_processes) as pool:
            results = []
            with tqdm(total=len(text_args), desc=f"TXT 처리 ({folder_name})") as pbar:
                for result in pool.imap(process_single_text_file, text_args):
                    results.append(result)
                    pbar.update(1)
        
        # 결과 집계
        for result in results:
            if result['status'] == 'success':
                text_success += 1
            elif result['status'] == 'skip':
                text_skip += 1
            else:
                text_error += 1
                print(f"    ❌ 텍스트 오류 ({result['file']}): {result.get('error', '알 수 없음')}")
    
    return {
        'folder': folder_name,
        'pcm_files': len(pcm_files),
        'txt_files': len(txt_files),
        'audio_success': audio_success,
        'audio_skip': audio_skip,
        'audio_error': audio_error,
        'text_success': text_success,
        'text_skip': text_skip,
        'text_error': text_error
    }

# 메인 전처리 함수 (멀티프로세싱 적용)
def preprocess_kspon_data_mp(num_processes=None, start_from=None, max_folders=None):
    """KsponSpeech_04 모든 폴더의 데이터를 멀티프로세싱으로 전처리합니다."""
    
    # CPU 코어 수 확인
    total_cores = mp.cpu_count()
    if num_processes is None:
        num_processes = min(total_cores, 8)
    
    print(f"🖥️ 시스템 정보:")
    print(f"  - 총 CPU 코어: {total_cores}개")
    print(f"  - 사용할 프로세스: {num_processes}개")
    
    # 현재 디렉토리에서 KsponSpeech_로 시작하는 모든 폴더 찾기
    all_folders = sorted([d for d in os.listdir('.') 
                         if d.startswith('KsponSpeech_') and os.path.isdir(d)])
    
    # 시작 지점 설정
    if start_from:
        try:
            start_idx = all_folders.index(start_from)
            all_folders = all_folders[start_idx:]
            print(f"📍 {start_from}부터 재시작")
        except ValueError:
            print(f"⚠️ {start_from} 폴더를 찾을 수 없습니다. 처음부터 시작합니다.")
    
    # 폴더 수 제한
    if max_folders:
        all_folders = all_folders[:max_folders]
        print(f"📊 최대 {max_folders}개 폴더만 처리")
    
    print(f"📁 처리할 폴더: {len(all_folders)}개")
    
    # 저장할 디렉토리들 생성
    audio_target_dir = f"{target_base_dir}/audio_features"
    text_target_dir = f"{target_base_dir}/g2p_texts"
    
    print(f"\n💾 저장 위치: {target_base_dir}")
    print(f"  - 오디오 특성 (NPZ): {audio_target_dir}")
    print(f"  - G2P 텍스트: {text_target_dir}")
    
    os.makedirs(audio_target_dir, exist_ok=True)
    os.makedirs(text_target_dir, exist_ok=True)
    
    # 전체 통계
    total_results = []
    start_time = time.time()
    
    print("\n" + "="*80)
    print("🚀 KsponSpeech_04 멀티프로세싱 전처리 시작")
    print("="*80)
    
    # 각 폴더 처리
    for i, folder in enumerate(all_folders, 1):
        source_dir = f"./{folder}"
        
        print(f"\n[{i}/{len(all_folders)}] 📂 {folder} 처리 중...")
        
        if not os.path.exists(source_dir):
            print(f"⚠️ 소스 디렉토리가 존재하지 않습니다: {source_dir}")
            continue
        
        folder_start_time = time.time()
        
        # 단일 폴더 멀티프로세싱 처리
        result = process_single_folder_mp(source_dir, audio_target_dir, text_target_dir, num_processes)
        total_results.append(result)
        
        folder_time = time.time() - folder_start_time
        
        print(f"  ✅ {folder} 완료 (소요시간: {folder_time:.1f}초):")
        print(f"    📊 파일 수: PCM {result['pcm_files']}개, TXT {result['txt_files']}개")
        print(f"    🎵 오디오: 성공 {result['audio_success']}, 건너뜀 {result['audio_skip']}, 오류 {result['audio_error']}")
        print(f"    📝 텍스트: 성공 {result['text_success']}, 건너뜀 {result['text_skip']}, 오류 {result['text_error']}")
        
        # 예상 남은 시간 계산
        if i < len(all_folders):
            elapsed = time.time() - start_time
            avg_time_per_folder = elapsed / i
            remaining_folders = len(all_folders) - i
            estimated_remaining = avg_time_per_folder * remaining_folders
            print(f"    ⏱️ 예상 남은 시간: {estimated_remaining/3600:.1f}시간")
    
    total_time = time.time() - start_time
    
    print("\n" + "="*80)
    print("🎉 전체 처리 완료!")
    print("="*80)
    
    # 전체 통계 계산
    total_audio_success = sum(r['audio_success'] for r in total_results)
    total_audio_skip = sum(r['audio_skip'] for r in total_results)
    total_audio_error = sum(r['audio_error'] for r in total_results)
    total_text_success = sum(r['text_success'] for r in total_results)
    total_text_skip = sum(r['text_skip'] for r in total_results)
    total_text_error = sum(r['text_error'] for r in total_results)
    
    print(f"\n📈 전체 통계:")
    print(f"  - 처리된 폴더: {len(total_results)}개")
    print(f"  - 총 소요시간: {total_time/3600:.1f}시간")
    print(f"  - 평균 폴더당 시간: {total_time/len(total_results)/60:.1f}분")
    print(f"\n🎵 오디오 파일:")
    print(f"  - 성공: {total_audio_success:,}개")
    print(f"  - 건너뜀: {total_audio_skip:,}개")
    print(f"  - 오류: {total_audio_error:,}개")
    print(f"\n📝 텍스트 파일:")
    print(f"  - 성공: {total_text_success:,}개")
    print(f"  - 건너뜀: {total_text_skip:,}개")
    print(f"  - 오류: {total_text_error:,}개")
    
    # 최종 파일 수 확인
    try:
        processed_audio_files = len([f for f in os.listdir(audio_target_dir) if f.endswith('.npz')])
        processed_text_files = len([f for f in os.listdir(text_target_dir) if f.endswith('.txt')])
        
        print(f"\n📦 최종 결과:")
        print(f"  - 저장된 오디오 파일: {processed_audio_files:,}개")
        print(f"  - 저장된 텍스트 파일: {processed_text_files:,}개")
        
        if processed_audio_files == processed_text_files:
            print("  ✅ 오디오와 텍스트 파일 수가 일치합니다!")
        else:
            print("  ⚠️ 오디오와 텍스트 파일 수가 일치하지 않습니다.")
            
    except Exception as e:
        print(f"❌ 최종 파일 수 확인 중 오류: {str(e)}")
    
    return total_results

# 실행 함수들
def run_full_processing():
    """전체 폴더 처리"""
    print("🚀 전체 폴더 멀티프로세싱 시작")
    return preprocess_kspon_data_mp()

def run_partial_processing(max_folders=20):
    """일부 폴더만 처리 (테스트용)"""
    print(f"🧪 {max_folders}개 폴더만 처리 (테스트)")
    return preprocess_kspon_data_mp(max_folders=max_folders)

def run_resume_processing(start_from="KsponSpeech_0380"):
    """특정 폴더부터 재시작"""
    print(f"🔄 {start_from}부터 재시작")
    return preprocess_kspon_data_mp(start_from=start_from)

# 메인 실행부
if __name__ == "__main__":
    print("🎯 KsponSpeech_04 멀티프로세싱 전처리")
    print("선택하세요:")
    print("1. 전체 처리 - run_full_processing()")
    print("2. 테스트 (20개) - run_partial_processing(20)")
    print("3. 재시작 - run_resume_processing('KsponSpeech_0392')")
    
    # 기본 실행: 재시작 모드
    run_resume_processing("KsponSpeech_0392")

🎯 KsponSpeech_04 멀티프로세싱 전처리
선택하세요:
1. 전체 처리 - run_full_processing()
2. 테스트 (20개) - run_partial_processing(20)
3. 재시작 - run_resume_processing('KsponSpeech_0392')
🔄 KsponSpeech_0392부터 재시작
🖥️ 시스템 정보:
  - 총 CPU 코어: 48개
  - 사용할 프로세스: 8개
📍 KsponSpeech_0392부터 재시작
📁 처리할 폴더: 105개

💾 저장 위치: ./PreprocessData_04
  - 오디오 특성 (NPZ): ./PreprocessData_04/audio_features
  - G2P 텍스트: ./PreprocessData_04/g2p_texts

🚀 KsponSpeech_04 멀티프로세싱 전처리 시작

[1/105] 📂 KsponSpeech_0392 처리 중...
🔄 멀티프로세싱 시작: 8개 프로세스 사용
  🎵 PCM 파일 1000개 병렬 처리 중...


PCM 처리 (KsponSpeech_0392): 100%|█████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 32585.72it/s]

  📝 TXT 파일 1000개 병렬 처리 중...



TXT 처리 (KsponSpeech_0392): 100%|█████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 30476.10it/s]

  ✅ KsponSpeech_0392 완료 (소요시간: 0.2초):
    📊 파일 수: PCM 1000개, TXT 1000개
    🎵 오디오: 성공 0, 건너뜀 1000, 오류 0
    📝 텍스트: 성공 0, 건너뜀 1000, 오류 0
    ⏱️ 예상 남은 시간: 0.0시간

[2/105] 📂 KsponSpeech_0393 처리 중...
🔄 멀티프로세싱 시작: 8개 프로세스 사용
  🎵 PCM 파일 1000개 병렬 처리 중...


...


PCM 처리 (KsponSpeech_0470): 100%|████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [04:16<00:00,  3.90it/s]

  📝 TXT 파일 1000개 병렬 처리 중...



TXT 처리 (KsponSpeech_0470): 100%|████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:26<00:00, 11.62it/s]

  ✅ KsponSpeech_0470 완료 (소요시간: 342.9초):
    📊 파일 수: PCM 1000개, TXT 1000개
    🎵 오디오: 성공 1000, 건너뜀 0, 오류 0
    📝 텍스트: 성공 1000, 건너뜀 0, 오류 0
    ⏱️ 예상 남은 시간: 1.9시간

[80/105] 📂 KsponSpeech_0471 처리 중...
🔄 멀티프로세싱 시작: 8개 프로세스 사용
  🎵 PCM 파일 1000개 병렬 처리 중...



PCM 처리 (KsponSpeech_0471):   3%|██▍                                                                                   | 29/1000 [00:11<04:46,  3.39it/s]

In [9]:
!find "g2p_texts -type f | wc -l

SyntaxError: invalid syntax (2542020279.py, line 1)

In [8]:
cd PreprocessData_04

/data/Traindata_2/KsponSpeech_04/PreprocessData_04


In [3]:
#!/usr/bin/env python3
"""
Whisper Fine-tuning for Korean STT
Dataset: 124,000 KsponSpeech samples
"""

import os
import json
import torch
import numpy as np
import pandas as pd
from pathlib import Path
from datasets import Dataset, DatasetDict  # load_metric 제거
from transformers import (
    WhisperFeatureExtractor,
    WhisperTokenizer,
    WhisperProcessor,
    WhisperForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    EarlyStoppingCallback
)
import librosa
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import evaluate  # 새로 추가
import warnings
warnings.filterwarnings("ignore")

# =============================================================================
# 설정 및 경로
# =============================================================================

class Config:
    """학습 설정"""
    
    # 데이터 경로
    DATA_DIR = "/path/to/your/processed_data"  # 실제 경로로 수정 필요
    OUTPUT_DIR = "./whisper-korean-finetuned"
    
    # 모델 설정
    MODEL_NAME = "openai/whisper-small"  # small, base, large 중 선택
    LANGUAGE = "korean"
    TASK = "transcribe"
    
    # 데이터셋 설정
    TRAIN_SIZE = 48000    # 6만개의 80%
    VAL_SIZE = 6000       # 6만개의 10%
    TEST_SIZE = 6000      # 6만개의 10%
    NUM_EPOCHS = 3  
    
    # 학습 하이퍼파라미터
    BATCH_SIZE = 8       # GPU 메모리에 따라 조정
    GRADIENT_ACCUMULATION_STEPS = 4
    LEARNING_RATE = 1e-5
    NUM_EPOCHS = 3
    WARMUP_STEPS = 500
    
    # 체크포인트 설정
    SAVE_STEPS = 1000
    EVAL_STEPS = 500
    SAVE_TOTAL_LIMIT = 3
    
    # 하드웨어 설정
    DATALOADER_NUM_WORKERS = 4
    USE_FP16 = True  # Mixed precision

config = Config()

# =============================================================================
# 데이터 로딩 및 전처리
# =============================================================================

def load_dataset_from_directory(data_dir: str) -> DatasetDict:
    """
    실제 전처리된 데이터 구조에서 데이터셋 로드
    - 텍스트: g2p_texts/*.txt
    - 오디오 특성: audio_features/*.npy 또는 *.npz
    """
    print("📂 데이터셋 로딩 중...")
    
    # 실제 경로 설정
    base_dir = Path(data_dir)
    text_dir = base_dir / "g2p_texts"
    audio_dir = base_dir / "audio_features"
    
    print(f"📁 텍스트 디렉토리: {text_dir}")
    print(f"🎵 오디오 디렉토리: {audio_dir}")
    
    # 디렉토리 존재 확인
    if not text_dir.exists():
        raise FileNotFoundError(f"텍스트 디렉토리가 없습니다: {text_dir}")
    if not audio_dir.exists():
        raise FileNotFoundError(f"오디오 디렉토리가 없습니다: {audio_dir}")
    
    # 매칭되는 파일 쌍 찾기
    audio_files = []
    transcripts = []
    
    # 텍스트 파일 기준으로 매칭
    text_files = list(text_dir.glob("*.txt"))
    print(f"📝 텍스트 파일 수: {len(text_files):,}개")
    
    matched_count = 0
    for text_file in text_files:
        # 대응하는 오디오 특성 파일 찾기
        base_name = text_file.stem
        
        # .npy 또는 .npz 파일 찾기
        audio_npy = audio_dir / f"{base_name}.npy"
        audio_npz = audio_dir / f"{base_name}.npz"
        
        audio_file = None
        if audio_npy.exists():
            audio_file = audio_npy
        elif audio_npz.exists():
            audio_file = audio_npz
        
        if audio_file:
            # 텍스트 읽기
            try:
                with open(text_file, 'r', encoding='utf-8') as f:
                    transcript = f.read().strip()
                
                if transcript:  # 빈 텍스트 제외
                    audio_files.append(str(audio_file))
                    transcripts.append(transcript)
                    matched_count += 1
                    
            except Exception as e:
                print(f"⚠️ 텍스트 파일 읽기 실패: {text_file}, 오류: {e}")
                continue
        
        # 진행 상황 출력
        if matched_count % 10000 == 0 and matched_count > 0:
            print(f"📊 진행 상황: {matched_count:,}개 매칭됨")
    
    print(f"✅ 총 {len(audio_files):,}개 파일 쌍 매칭됨")
    
    if len(audio_files) == 0:
        raise ValueError("매칭되는 파일이 없습니다. 경로와 파일 형식을 확인해주세요.")
    
    # 데이터프레임 생성
    df = pd.DataFrame({
        'audio_path': audio_files,
        'transcript': transcripts
    })
    
    # 데이터 검증
    print(f"📊 데이터 검증:")
    print(f"  - 평균 텍스트 길이: {df['transcript'].str.len().mean():.1f}자")
    print(f"  - 최대 텍스트 길이: {df['transcript'].str.len().max()}자")
    print(f"  - 빈 텍스트: {df['transcript'].str.len().eq(0).sum()}개")
    
    # 빈 텍스트 제거
    df = df[df['transcript'].str.len() > 0].reset_index(drop=True)
    print(f"  - 유효한 데이터: {len(df):,}개")
    
    # 데이터 분할 (사용 가능한 데이터에 맞춰 조정)
    total_size = len(df)
    actual_train_size = min(config.TRAIN_SIZE, int(total_size * 0.8))
    actual_val_size = min(config.VAL_SIZE, int(total_size * 0.1))
    actual_test_size = min(config.TEST_SIZE, int(total_size * 0.1))
    
    # 데이터 셔플
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)
    
    train_df = df[:actual_train_size]
    val_df = df[actual_train_size:actual_train_size + actual_val_size]
    test_df = df[actual_train_size + actual_val_size:actual_train_size + actual_val_size + actual_test_size]
    
    print(f"📊 최종 데이터 분할:")
    print(f"  - 학습: {len(train_df):,}개")
    print(f"  - 검증: {len(val_df):,}개") 
    print(f"  - 테스트: {len(test_df):,}개")
    
    # Hugging Face Dataset으로 변환
    train_dataset = Dataset.from_pandas(train_df)
    val_dataset = Dataset.from_pandas(val_df)
    test_dataset = Dataset.from_pandas(test_df)
    
    return DatasetDict({
        "train": train_dataset,
        "validation": val_dataset,
        "test": test_dataset
    })

def load_audio_features(audio_path: str) -> np.ndarray:
    """
    전처리된 오디오 특성 파일 로드 (.npy 또는 .npz)
    """
    try:
        if audio_path.endswith('.npy'):
            # .npy 파일 로드
            features = np.load(audio_path)
        elif audio_path.endswith('.npz'):
            # .npz 파일 로드
            data = np.load(audio_path)
            # 첫 번째 키의 데이터 사용 (또는 특정 키 지정)
            key = list(data.keys())[0]
            features = data[key]
        else:
            raise ValueError(f"지원하지 않는 파일 형식: {audio_path}")
        
        # 특성이 올바른 형태인지 확인
        if features.ndim == 1:
            # 1D 배열이면 2D로 변환 (time_steps, features)
            features = features.reshape(-1, 1)
        elif features.ndim == 2:
            # 2D 배열이면 그대로 사용
            pass
        else:
            print(f"⚠️ 예상치 못한 특성 형태: {features.shape} in {audio_path}")
        
        return features
        
    except Exception as e:
        print(f"오디오 특성 로드 실패: {audio_path}, 오류: {e}")
        # 기본 특성 반환 (에러 방지)
        return np.zeros((1, 80))  # Whisper 멜 스펙트로그램 차원

def load_audio(audio_path: str, target_sr: int = 16000) -> np.ndarray:
    """
    오디오 파일 로드 (WAV 파일용 - 실제로는 특성 파일을 로드)
    """
    # 실제로는 전처리된 특성 파일을 로드
    return load_audio_features(audio_path)

# =============================================================================
# 모델 및 프로세서 초기화  
# =============================================================================

def initialize_model_and_processor():
    """Whisper 모델과 프로세서 초기화"""
    print("🤖 모델 초기화 중...")
    
    # 프로세서 구성요소
    feature_extractor = WhisperFeatureExtractor.from_pretrained(config.MODEL_NAME)
    tokenizer = WhisperTokenizer.from_pretrained(
        config.MODEL_NAME, 
        language=config.LANGUAGE,
        task=config.TASK
    )
    processor = WhisperProcessor.from_pretrained(
        config.MODEL_NAME,
        language=config.LANGUAGE,
        task=config.TASK
    )
    
    # 모델 로드
    model = WhisperForConditionalGeneration.from_pretrained(config.MODEL_NAME)
    
    # 언어 토큰 설정
    model.config.forced_decoder_ids = None
    model.config.suppress_tokens = []
    
    # 한국어 설정
    model.generation_config.language = config.LANGUAGE
    model.generation_config.task = config.TASK
    
    print(f"✅ 모델 로드 완료: {config.MODEL_NAME}")
    
    return model, processor, feature_extractor, tokenizer

# =============================================================================
# 데이터 전처리 및 콜레이터
# =============================================================================

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    """
    음성-텍스트 데이터 콜레이터
    """
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # 오디오 입력 패딩
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # 레이블 패딩
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # -100으로 패딩된 토큰을 마스킹 (손실 계산에서 제외)
        labels = labels_batch["input_ids"].masked_fill(
            labels_batch.attention_mask.ne(1), -100
        )

        # 시작 토큰이 있다면 제거 (모델이 자동으로 추가)
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels
        return batch

def prepare_dataset(batch, processor, feature_extractor):
    """
    전처리된 특성 데이터를 위한 데이터셋 준비 함수
    """
    
    # 전처리된 오디오 특성 로드
    audio_features = load_audio_features(batch["audio_path"])
    
    # 특성 형태 확인 및 조정
    if audio_features.ndim == 2:
        # (time_steps, features) → (features, time_steps) 변환이 필요할 수 있음
        if audio_features.shape[1] > audio_features.shape[0]:
            # 일반적으로 (features, time_steps) 형태가 맞음
            pass
        else:
            # (time_steps, features) → (features, time_steps)
            audio_features = audio_features.T
    
    # Whisper input_features 형태로 변환
    # Whisper는 (80, 3000) 형태의 멜 스펙트로그램을 기대
    target_time_steps = 3000  # 30초 * 100 (10ms 프레임)
    
    if audio_features.shape[1] > target_time_steps:
        # 너무 긴 경우 자르기
        audio_features = audio_features[:, :target_time_steps]
    elif audio_features.shape[1] < target_time_steps:
        # 너무 짧은 경우 패딩
        pad_width = target_time_steps - audio_features.shape[1]
        audio_features = np.pad(audio_features, ((0, 0), (0, pad_width)), mode='constant', constant_values=0)
    
    # 특성 차원 조정 (80차원으로 맞추기)
    if audio_features.shape[0] != 80:
        if audio_features.shape[0] < 80:
            # 부족한 차원 패딩
            pad_features = 80 - audio_features.shape[0]
            audio_features = np.pad(audio_features, ((0, pad_features), (0, 0)), mode='constant', constant_values=0)
        else:
            # 초과 차원 자르기
            audio_features = audio_features[:80, :]
    
    # input_features로 설정
    batch["input_features"] = audio_features.astype(np.float32)
    
    # 텍스트 토큰화
    batch["labels"] = processor.tokenizer(
        batch["transcript"],
        truncation=True,
        max_length=448,  # Whisper 최대 길이
        padding=False,
        return_tensors="pt"
    ).input_ids[0]
    
    return batch

# =============================================================================
# 평가 메트릭
# =============================================================================

def compute_metrics(eval_pred, tokenizer):
    """WER 및 CER 계산"""
    pred_ids, label_ids = eval_pred
    
    # -100을 패딩 토큰으로 교체
    label_ids[label_ids == -100] = tokenizer.pad_token_id
    
    # 디코딩
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    
    # WER 계산 - evaluate 라이브러리 사용
    wer_metric = evaluate.load("wer")
    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    
    return {"wer": wer}

# =============================================================================
# 학습 실행
# =============================================================================

def main():
    """메인 학습 함수"""
    
    print("🚀 Whisper 파인튜닝 시작!")
    print(f"📊 데이터셋 크기: {config.TRAIN_SIZE + config.VAL_SIZE + config.TEST_SIZE:,}개")
    print(f"🎯 모델: {config.MODEL_NAME}")
    print(f"⚡ GPU: {'사용 가능' if torch.cuda.is_available() else '사용 불가'}")
    
    # 1. 데이터셋 로드
    dataset = load_dataset_from_directory(config.DATA_DIR)
    
    # 2. 모델 초기화
    model, processor, feature_extractor, tokenizer = initialize_model_and_processor()
    
    # 3. 데이터 전처리
    print("🔄 데이터 전처리 중...")
    dataset = dataset.map(
        lambda batch: prepare_dataset(batch, processor, feature_extractor),
        remove_columns=dataset["train"].column_names,
        num_proc=config.DATALOADER_NUM_WORKERS
    )
    
    # 4. 데이터 콜레이터 설정
    data_collator = DataCollatorSpeechSeq2SeqWithPadding(
        processor=processor,
        decoder_start_token_id=model.config.decoder_start_token_id,
    )
    
    # 5. 학습 설정
    training_args = Seq2SeqTrainingArguments(
        output_dir=config.OUTPUT_DIR,
        per_device_train_batch_size=config.BATCH_SIZE,
        per_device_eval_batch_size=config.BATCH_SIZE,
        gradient_accumulation_steps=config.GRADIENT_ACCUMULATION_STEPS,
        learning_rate=config.LEARNING_RATE,
        num_train_epochs=config.NUM_EPOCHS,
        warmup_steps=config.WARMUP_STEPS,
        evaluation_strategy="steps",
        eval_steps=config.EVAL_STEPS,
        save_steps=config.SAVE_STEPS,
        save_total_limit=config.SAVE_TOTAL_LIMIT,
        logging_steps=100,
        remove_unused_columns=False,
        label_names=["labels"],
        load_best_model_at_end=True,
        metric_for_best_model="wer",
        greater_is_better=False,
        push_to_hub=False,
        dataloader_num_workers=config.DATALOADER_NUM_WORKERS,
        fp16=config.USE_FP16,
        gradient_checkpointing=True,  # 메모리 절약
        report_to=["tensorboard"],
    )
    
    # 6. 트레이너 설정
    trainer = Seq2SeqTrainer(
        args=training_args,
        model=model,
        train_dataset=dataset["train"],
        eval_dataset=dataset["validation"],
        data_collator=data_collator,
        compute_metrics=lambda eval_pred: compute_metrics(eval_pred, tokenizer),
        tokenizer=processor.feature_extractor,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )
    
    # 7. 학습 시작
    print("🎓 학습 시작!")
    trainer.train()
    
    # 8. 모델 저장
    print("💾 모델 저장 중...")
    trainer.save_model()
    processor.save_pretrained(config.OUTPUT_DIR)
    
    # 9. 테스트 세트 평가
    print("📊 테스트 세트 평가 중...")
    test_results = trainer.evaluate(dataset["test"])
    print(f"🎯 최종 테스트 WER: {test_results['eval_wer']:.4f}")
    
    # 10. 결과 저장
    results = {
        "model_name": config.MODEL_NAME,
        "dataset_size": len(dataset["train"]) + len(dataset["validation"]),
        "test_wer": test_results['eval_wer'],
        "config": config.__dict__
    }
    
    with open(os.path.join(config.OUTPUT_DIR, "training_results.json"), "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)
    
    print("🎉 파인튜닝 완료!")
    print(f"📁 모델 저장 위치: {config.OUTPUT_DIR}")

# =============================================================================
# 간단한 추론 테스트
# =============================================================================

def test_inference(model_path: str, audio_path: str):
    """학습된 모델로 추론 테스트"""
    
    # 모델 로드
    model = WhisperForConditionalGeneration.from_pretrained(model_path)
    processor = WhisperProcessor.from_pretrained(model_path)
    
    # 오디오 로드
    audio = load_audio(audio_path)
    
    # 특성 추출
    input_features = processor(
        audio, 
        sampling_rate=16000, 
        return_tensors="pt"
    ).input_features
    
    # 추론
    with torch.no_grad():
        predicted_ids = model.generate(input_features)
    
    # 디코딩
    transcription = processor.batch_decode(
        predicted_ids, 
        skip_special_tokens=True
    )[0]
    
    print(f"🎤 음성 인식 결과: {transcription}")
    return transcription

if __name__ == "__main__":
    # 실제 데이터 경로 설정
    default_path = "Traindata_2/KsponSpeech_04/PreprocessData_04"
    
    data_path = input(f"📂 데이터 디렉토리 경로 (기본값: {default_path}): ").strip()
    if not data_path:
        data_path = default_path
    
    config.DATA_DIR = data_path
    
    # 경로 검증
    text_dir = Path(data_path) / "g2p_texts"
    audio_dir = Path(data_path) / "audio_features"
    
    print(f"\n📁 설정된 경로:")
    print(f"  - 기본 경로: {data_path}")
    print(f"  - 텍스트: {text_dir}")
    print(f"  - 오디오: {audio_dir}")
    
    if not text_dir.exists():
        print(f"❌ 텍스트 디렉토리가 존재하지 않습니다: {text_dir}")
        exit(1)
    
    if not audio_dir.exists():
        print(f"❌ 오디오 디렉토리가 존재하지 않습니다: {audio_dir}")
        exit(1)
    
    # 파일 수 미리 확인
    text_files = list(text_dir.glob("*.txt"))
    audio_npy_files = list(audio_dir.glob("*.npy"))
    audio_npz_files = list(audio_dir.glob("*.npz"))
    audio_files = audio_npy_files + audio_npz_files
    
    print(f"\n📊 파일 수 확인:")
    print(f"  - 텍스트 파일: {len(text_files):,}개")
    print(f"  - 오디오 파일: {len(audio_files):,}개 (.npy: {len(audio_npy_files)}, .npz: {len(audio_npz_files)})")
    
    if len(text_files) == 0 or len(audio_files) == 0:
        print("❌ 처리할 파일이 없습니다!")
        exit(1)
    
    # 확인 후 진행
    proceed = input(f"\n🚀 {min(len(text_files), len(audio_files)):,}개 파일로 학습을 시작하시겠습니까? (y/N): ").strip().lower()
    if proceed != 'y':
        print("학습을 중단합니다.")
        exit(0)
    
    # 학습 실행
    main()
    
    # 테스트 (옵션)
    test_audio = input("\n🎵 테스트할 오디오 특성 파일 경로 (선택사항, Enter로 건너뛰기): ").strip()
    if test_audio and os.path.exists(test_audio):
        try:
            test_inference(config.OUTPUT_DIR, test_audio)
        except Exception as e:
            print(f"⚠️ 추론 테스트 실패: {e}")

📂 데이터 디렉토리 경로 (기본값: Traindata_2/KsponSpeech_04/PreprocessData_04):  KsponSpeech_04/PreprocessData_04



📁 설정된 경로:
  - 기본 경로: KsponSpeech_04/PreprocessData_04
  - 텍스트: KsponSpeech_04/PreprocessData_04/g2p_texts
  - 오디오: KsponSpeech_04/PreprocessData_04/audio_features

📊 파일 수 확인:
  - 텍스트 파일: 124,000개
  - 오디오 파일: 124,000개 (.npy: 0, .npz: 124000)



🚀 124,000개 파일로 학습을 시작하시겠습니까? (y/N):  y


🚀 Whisper 파인튜닝 시작!
📊 데이터셋 크기: 60,000개
🎯 모델: openai/whisper-small
⚡ GPU: 사용 가능
📂 데이터셋 로딩 중...
📁 텍스트 디렉토리: KsponSpeech_04/PreprocessData_04/g2p_texts
🎵 오디오 디렉토리: KsponSpeech_04/PreprocessData_04/audio_features
📝 텍스트 파일 수: 124,000개
📊 진행 상황: 10,000개 매칭됨
📊 진행 상황: 20,000개 매칭됨
📊 진행 상황: 30,000개 매칭됨
📊 진행 상황: 40,000개 매칭됨
📊 진행 상황: 50,000개 매칭됨
📊 진행 상황: 60,000개 매칭됨
📊 진행 상황: 70,000개 매칭됨
📊 진행 상황: 80,000개 매칭됨
📊 진행 상황: 90,000개 매칭됨
📊 진행 상황: 100,000개 매칭됨
📊 진행 상황: 110,000개 매칭됨
📊 진행 상황: 120,000개 매칭됨
✅ 총 124,000개 파일 쌍 매칭됨
📊 데이터 검증:
  - 평균 텍스트 길이: 39.8자
  - 최대 텍스트 길이: 384자
  - 빈 텍스트: 0개
  - 유효한 데이터: 124,000개
📊 최종 데이터 분할:
  - 학습: 48,000개
  - 검증: 6,000개
  - 테스트: 6,000개
🤖 모델 초기화 중...
✅ 모델 로드 완료: openai/whisper-small
🔄 데이터 전처리 중...


Map (num_proc=4): 100%|█████████████████████████████████████████████████████████████████████████████████████| 48000/48000 [01:32<00:00, 517.16 examples/s]


RuntimeError: One of the subprocesses has abruptly died during map operation.To debug the error, disable multiprocessing.

In [13]:
pip install evaluate

     |████████████████████████████████| 84 kB 4.7 MB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
ls

KsponSpeech_04/  PreprocessData_04/  Untitled.ipynb


In [4]:
#!/usr/bin/env python3
"""
Whisper Fine-tuning for Korean STT
Dataset: 124,000 KsponSpeech samples
"""

import os
import json
import torch
import numpy as np
import pandas as pd
from pathlib import Path
from datasets import Dataset, DatasetDict  # load_metric 제거
from transformers import (
    WhisperFeatureExtractor,
    WhisperTokenizer,
    WhisperProcessor,
    WhisperForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    EarlyStoppingCallback
)
import librosa
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import evaluate  # 새로 추가
import warnings
warnings.filterwarnings("ignore")

# =============================================================================
# 설정 및 경로
# =============================================================================

class Config:
    """학습 설정"""
    
    # 데이터 경로
    DATA_DIR = "/path/to/your/processed_data"  # 실제 경로로 수정 필요
    OUTPUT_DIR = "./whisper-korean-finetuned"
    
    # 모델 설정
    MODEL_NAME = "openai/whisper-small"  # small, base, large 중 선택
    LANGUAGE = "korean"
    TASK = "transcribe"
    
    # 데이터셋 설정
    TRAIN_SIZE = 48000    # 6만개의 80%
    VAL_SIZE = 6000       # 6만개의 10% 
    TEST_SIZE = 6000      # 6만개의 10%
    MAX_INPUT_LENGTH = 40.0  # 40초
    
    # 학습 하이퍼파라미터
    BATCH_SIZE = 8        # GPU 메모리에 따라 조정 (16→8로 줄임)
    GRADIENT_ACCUMULATION_STEPS = 4  # 2→4로 늘림 (실효 배치 크기 유지)
    LEARNING_RATE = 1e-5
    NUM_EPOCHS = 3        # 5→3으로 줄임
    WARMUP_STEPS = 500
    
    # 체크포인트 설정
    SAVE_STEPS = 1000
    EVAL_STEPS = 500
    SAVE_TOTAL_LIMIT = 3
    
    # 하드웨어 설정
    DATALOADER_NUM_WORKERS = 0  # 멀티프로세싱 비활성화
    USE_FP16 = True  # Mixed precision

config = Config()

# =============================================================================
# 데이터 로딩 및 전처리
# =============================================================================

def load_dataset_from_directory(data_dir: str) -> DatasetDict:
    """
    실제 전처리된 데이터 구조에서 데이터셋 로드
    - 텍스트: g2p_texts/*.txt
    - 오디오 특성: audio_features/*.npy 또는 *.npz
    """
    print("📂 데이터셋 로딩 중...")
    
    # 실제 경로 설정
    base_dir = Path(data_dir)
    text_dir = base_dir / "g2p_texts"
    audio_dir = base_dir / "audio_features"
    
    print(f"📁 텍스트 디렉토리: {text_dir}")
    print(f"🎵 오디오 디렉토리: {audio_dir}")
    
    # 디렉토리 존재 확인
    if not text_dir.exists():
        raise FileNotFoundError(f"텍스트 디렉토리가 없습니다: {text_dir}")
    if not audio_dir.exists():
        raise FileNotFoundError(f"오디오 디렉토리가 없습니다: {audio_dir}")
    
    # 매칭되는 파일 쌍 찾기
    audio_files = []
    transcripts = []
    
    # 텍스트 파일 기준으로 매칭
    text_files = list(text_dir.glob("*.txt"))
    print(f"📝 텍스트 파일 수: {len(text_files):,}개")
    
    matched_count = 0
    for text_file in text_files:
        # 대응하는 오디오 특성 파일 찾기
        base_name = text_file.stem
        
        # .npy 또는 .npz 파일 찾기
        audio_npy = audio_dir / f"{base_name}.npy"
        audio_npz = audio_dir / f"{base_name}.npz"
        
        audio_file = None
        if audio_npy.exists():
            audio_file = audio_npy
        elif audio_npz.exists():
            audio_file = audio_npz
        
        if audio_file:
            # 텍스트 읽기
            try:
                with open(text_file, 'r', encoding='utf-8') as f:
                    transcript = f.read().strip()
                
                if transcript:  # 빈 텍스트 제외
                    audio_files.append(str(audio_file))
                    transcripts.append(transcript)
                    matched_count += 1
                    
            except Exception as e:
                print(f"⚠️ 텍스트 파일 읽기 실패: {text_file}, 오류: {e}")
                continue
        
        # 진행 상황 출력
        if matched_count % 10000 == 0 and matched_count > 0:
            print(f"📊 진행 상황: {matched_count:,}개 매칭됨")
    
    print(f"✅ 총 {len(audio_files):,}개 파일 쌍 매칭됨")
    
    if len(audio_files) == 0:
        raise ValueError("매칭되는 파일이 없습니다. 경로와 파일 형식을 확인해주세요.")
    
    # 데이터프레임 생성
    df = pd.DataFrame({
        'audio_path': audio_files,
        'transcript': transcripts
    })
    
    # 데이터 검증
    print(f"📊 데이터 검증:")
    print(f"  - 평균 텍스트 길이: {df['transcript'].str.len().mean():.1f}자")
    print(f"  - 최대 텍스트 길이: {df['transcript'].str.len().max()}자")
    print(f"  - 빈 텍스트: {df['transcript'].str.len().eq(0).sum()}개")
    
    # 빈 텍스트 제거
    df = df[df['transcript'].str.len() > 0].reset_index(drop=True)
    print(f"  - 유효한 데이터: {len(df):,}개")
    
    # 데이터 분할 (사용 가능한 데이터에 맞춰 조정)
    total_size = len(df)
    actual_train_size = min(config.TRAIN_SIZE, int(total_size * 0.8))
    actual_val_size = min(config.VAL_SIZE, int(total_size * 0.1))
    actual_test_size = min(config.TEST_SIZE, int(total_size * 0.1))
    
    # 데이터 셔플
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)
    
    train_df = df[:actual_train_size]
    val_df = df[actual_train_size:actual_train_size + actual_val_size]
    test_df = df[actual_train_size + actual_val_size:actual_train_size + actual_val_size + actual_test_size]
    
    print(f"📊 최종 데이터 분할:")
    print(f"  - 학습: {len(train_df):,}개")
    print(f"  - 검증: {len(val_df):,}개") 
    print(f"  - 테스트: {len(test_df):,}개")
    
    # Hugging Face Dataset으로 변환
    train_dataset = Dataset.from_pandas(train_df)
    val_dataset = Dataset.from_pandas(val_df)
    test_dataset = Dataset.from_pandas(test_df)
    
    return DatasetDict({
        "train": train_dataset,
        "validation": val_dataset,
        "test": test_dataset
    })

def load_audio_features(audio_path: str) -> np.ndarray:
    """
    전처리된 오디오 특성 파일 로드 (.npy 또는 .npz)
    """
    try:
        if audio_path.endswith('.npy'):
            # .npy 파일 로드
            features = np.load(audio_path)
        elif audio_path.endswith('.npz'):
            # .npz 파일 로드
            data = np.load(audio_path)
            # 첫 번째 키의 데이터 사용 (또는 특정 키 지정)
            key = list(data.keys())[0]
            features = data[key]
        else:
            raise ValueError(f"지원하지 않는 파일 형식: {audio_path}")
        
        # 특성이 올바른 형태인지 확인
        if features.ndim == 1:
            # 1D 배열이면 2D로 변환 (time_steps, features)
            features = features.reshape(-1, 1)
        elif features.ndim == 2:
            # 2D 배열이면 그대로 사용
            pass
        else:
            print(f"⚠️ 예상치 못한 특성 형태: {features.shape} in {audio_path}")
        
        return features
        
    except Exception as e:
        print(f"오디오 특성 로드 실패: {audio_path}, 오류: {e}")
        # 기본 특성 반환 (에러 방지)
        return np.zeros((1, 80))  # Whisper 멜 스펙트로그램 차원

def load_audio(audio_path: str, target_sr: int = 16000) -> np.ndarray:
    """
    오디오 파일 로드 (WAV 파일용 - 실제로는 특성 파일을 로드)
    """
    # 실제로는 전처리된 특성 파일을 로드
    return load_audio_features(audio_path)

# =============================================================================
# 모델 및 프로세서 초기화  
# =============================================================================

def initialize_model_and_processor():
    """Whisper 모델과 프로세서 초기화"""
    print("🤖 모델 초기화 중...")
    
    # 프로세서 구성요소
    feature_extractor = WhisperFeatureExtractor.from_pretrained(config.MODEL_NAME)
    tokenizer = WhisperTokenizer.from_pretrained(
        config.MODEL_NAME, 
        language=config.LANGUAGE,
        task=config.TASK
    )
    processor = WhisperProcessor.from_pretrained(
        config.MODEL_NAME,
        language=config.LANGUAGE,
        task=config.TASK
    )
    
    # 모델 로드
    model = WhisperForConditionalGeneration.from_pretrained(config.MODEL_NAME)
    
    # 언어 토큰 설정
    model.config.forced_decoder_ids = None
    model.config.suppress_tokens = []
    
    # 한국어 설정
    model.generation_config.language = config.LANGUAGE
    model.generation_config.task = config.TASK
    
    print(f"✅ 모델 로드 완료: {config.MODEL_NAME}")
    
    return model, processor, feature_extractor, tokenizer

# =============================================================================
# 데이터 전처리 및 콜레이터
# =============================================================================

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    """
    음성-텍스트 데이터 콜레이터
    """
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # 오디오 입력 패딩
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # 레이블 패딩
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # -100으로 패딩된 토큰을 마스킹 (손실 계산에서 제외)
        labels = labels_batch["input_ids"].masked_fill(
            labels_batch.attention_mask.ne(1), -100
        )

        # 시작 토큰이 있다면 제거 (모델이 자동으로 추가)
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels
        return batch

def prepare_dataset(batch, processor, feature_extractor):
    """
    전처리된 특성 데이터를 위한 데이터셋 준비 함수
    """
    
    # 전처리된 오디오 특성 로드
    audio_features = load_audio_features(batch["audio_path"])
    
    # 특성 형태 확인 및 조정
    if audio_features.ndim == 2:
        # (time_steps, features) → (features, time_steps) 변환이 필요할 수 있음
        if audio_features.shape[1] > audio_features.shape[0]:
            # 일반적으로 (features, time_steps) 형태가 맞음
            pass
        else:
            # (time_steps, features) → (features, time_steps)
            audio_features = audio_features.T
    
    # Whisper input_features 형태로 변환
    # Whisper는 (80, 3000) 형태의 멜 스펙트로그램을 기대
    target_time_steps = 3000  # 30초 * 100 (10ms 프레임)
    
    if audio_features.shape[1] > target_time_steps:
        # 너무 긴 경우 자르기
        audio_features = audio_features[:, :target_time_steps]
    elif audio_features.shape[1] < target_time_steps:
        # 너무 짧은 경우 패딩
        pad_width = target_time_steps - audio_features.shape[1]
        audio_features = np.pad(audio_features, ((0, 0), (0, pad_width)), mode='constant', constant_values=0)
    
    # 특성 차원 조정 (80차원으로 맞추기)
    if audio_features.shape[0] != 80:
        if audio_features.shape[0] < 80:
            # 부족한 차원 패딩
            pad_features = 80 - audio_features.shape[0]
            audio_features = np.pad(audio_features, ((0, pad_features), (0, 0)), mode='constant', constant_values=0)
        else:
            # 초과 차원 자르기
            audio_features = audio_features[:80, :]
    
    # input_features로 설정
    batch["input_features"] = audio_features.astype(np.float32)
    
    # 텍스트 토큰화
    batch["labels"] = processor.tokenizer(
        batch["transcript"],
        truncation=True,
        max_length=448,  # Whisper 최대 길이
        padding=False,
        return_tensors="pt"
    ).input_ids[0]
    
    return batch

# =============================================================================
# 평가 메트릭
# =============================================================================

def compute_metrics(eval_pred, tokenizer):
    """WER 및 CER 계산"""
    pred_ids, label_ids = eval_pred
    
    # -100을 패딩 토큰으로 교체
    label_ids[label_ids == -100] = tokenizer.pad_token_id
    
    # 디코딩
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    
    # WER 계산 - evaluate 라이브러리 사용
    wer_metric = evaluate.load("wer")
    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    
    return {"wer": wer}

# =============================================================================
# 학습 실행
# =============================================================================

def main():
    """메인 학습 함수"""
    
    print("🚀 Whisper 파인튜닝 시작!")
    print(f"📊 데이터셋 크기: {config.TRAIN_SIZE + config.VAL_SIZE + config.TEST_SIZE:,}개")
    print(f"🎯 모델: {config.MODEL_NAME}")
    print(f"⚡ GPU: {'사용 가능' if torch.cuda.is_available() else '사용 불가'}")
    
    # 1. 데이터셋 로드
    dataset = load_dataset_from_directory(config.DATA_DIR)
    
    # 2. 모델 초기화
    model, processor, feature_extractor, tokenizer = initialize_model_and_processor()
    
    # 3. 데이터 전처리
    print("🔄 데이터 전처리 중...")
    dataset = dataset.map(
        lambda batch: prepare_dataset(batch, processor, feature_extractor),
        remove_columns=dataset["train"].column_names,
        num_proc=1  # 싱글 프로세스로 변경
    )
    
    # 4. 데이터 콜레이터 설정
    data_collator = DataCollatorSpeechSeq2SeqWithPadding(
        processor=processor,
        decoder_start_token_id=model.config.decoder_start_token_id,
    )
    
    # 5. 학습 설정
    training_args = Seq2SeqTrainingArguments(
        output_dir=config.OUTPUT_DIR,
        per_device_train_batch_size=config.BATCH_SIZE,
        per_device_eval_batch_size=config.BATCH_SIZE,
        gradient_accumulation_steps=config.GRADIENT_ACCUMULATION_STEPS,
        learning_rate=config.LEARNING_RATE,
        num_train_epochs=config.NUM_EPOCHS,
        warmup_steps=config.WARMUP_STEPS,
        evaluation_strategy="steps",
        eval_steps=config.EVAL_STEPS,
        save_steps=config.SAVE_STEPS,
        save_total_limit=config.SAVE_TOTAL_LIMIT,
        logging_steps=100,
        remove_unused_columns=False,
        label_names=["labels"],
        load_best_model_at_end=True,
        metric_for_best_model="wer",
        greater_is_better=False,
        push_to_hub=False,
        dataloader_num_workers=0,  # 멀티프로세싱 비활성화
        fp16=config.USE_FP16,
        gradient_checkpointing=True,  # 메모리 절약
        report_to=["tensorboard"],
    )
    
    # 6. 트레이너 설정
    trainer = Seq2SeqTrainer(
        args=training_args,
        model=model,
        train_dataset=dataset["train"],
        eval_dataset=dataset["validation"],
        data_collator=data_collator,
        compute_metrics=lambda eval_pred: compute_metrics(eval_pred, tokenizer),
        tokenizer=processor.feature_extractor,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )
    
    # 7. 학습 시작
    print("🎓 학습 시작!")
    trainer.train()
    
    # 8. 모델 저장
    print("💾 모델 저장 중...")
    trainer.save_model()
    processor.save_pretrained(config.OUTPUT_DIR)
    
    # 9. 테스트 세트 평가
    print("📊 테스트 세트 평가 중...")
    test_results = trainer.evaluate(dataset["test"])
    print(f"🎯 최종 테스트 WER: {test_results['eval_wer']:.4f}")
    
    # 10. 결과 저장
    results = {
        "model_name": config.MODEL_NAME,
        "dataset_size": len(dataset["train"]) + len(dataset["validation"]),
        "test_wer": test_results['eval_wer'],
        "config": config.__dict__
    }
    
    with open(os.path.join(config.OUTPUT_DIR, "training_results.json"), "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)
    
    print("🎉 파인튜닝 완료!")
    print(f"📁 모델 저장 위치: {config.OUTPUT_DIR}")

# =============================================================================
# 간단한 추론 테스트
# =============================================================================

def test_inference(model_path: str, audio_path: str):
    """학습된 모델로 추론 테스트"""
    
    # 모델 로드
    model = WhisperForConditionalGeneration.from_pretrained(model_path)
    processor = WhisperProcessor.from_pretrained(model_path)
    
    # 오디오 로드
    audio = load_audio(audio_path)
    
    # 특성 추출
    input_features = processor(
        audio, 
        sampling_rate=16000, 
        return_tensors="pt"
    ).input_features
    
    # 추론
    with torch.no_grad():
        predicted_ids = model.generate(input_features)
    
    # 디코딩
    transcription = processor.batch_decode(
        predicted_ids, 
        skip_special_tokens=True
    )[0]
    
    print(f"🎤 음성 인식 결과: {transcription}")
    return transcription

if __name__ == "__main__":
    # 실제 데이터 경로 설정
    default_path = "Traindata_2/KsponSpeech_04/PreprocessData_04"
    
    data_path = input(f"📂 데이터 디렉토리 경로 (기본값: {default_path}): ").strip()
    if not data_path:
        data_path = default_path
    
    config.DATA_DIR = data_path
    
    # 경로 검증
    text_dir = Path(data_path) / "g2p_texts"
    audio_dir = Path(data_path) / "audio_features"
    
    print(f"\n📁 설정된 경로:")
    print(f"  - 기본 경로: {data_path}")
    print(f"  - 텍스트: {text_dir}")
    print(f"  - 오디오: {audio_dir}")
    
    if not text_dir.exists():
        print(f"❌ 텍스트 디렉토리가 존재하지 않습니다: {text_dir}")
        exit(1)
    
    if not audio_dir.exists():
        print(f"❌ 오디오 디렉토리가 존재하지 않습니다: {audio_dir}")
        exit(1)
    
    # 파일 수 미리 확인
    text_files = list(text_dir.glob("*.txt"))
    audio_npy_files = list(audio_dir.glob("*.npy"))
    audio_npz_files = list(audio_dir.glob("*.npz"))
    audio_files = audio_npy_files + audio_npz_files
    
    print(f"\n📊 파일 수 확인:")
    print(f"  - 텍스트 파일: {len(text_files):,}개")
    print(f"  - 오디오 파일: {len(audio_files):,}개 (.npy: {len(audio_npy_files)}, .npz: {len(audio_npz_files)})")
    
    if len(text_files) == 0 or len(audio_files) == 0:
        print("❌ 처리할 파일이 없습니다!")
        exit(1)
    
    # 확인 후 진행
    proceed = input(f"\n🚀 {min(len(text_files), len(audio_files)):,}개 파일로 학습을 시작하시겠습니까? (y/N): ").strip().lower()
    if proceed != 'y':
        print("학습을 중단합니다.")
        exit(0)
    
    # 학습 실행
    main()
    
    # 테스트 (옵션)
    test_audio = input("\n🎵 테스트할 오디오 특성 파일 경로 (선택사항, Enter로 건너뛰기): ").strip()
    if test_audio and os.path.exists(test_audio):
        try:
            test_inference(config.OUTPUT_DIR, test_audio)
        except Exception as e:
            print(f"⚠️ 추론 테스트 실패: {e}")]

📂 데이터 디렉토리 경로 (기본값: Traindata_2/KsponSpeech_04/PreprocessData_04):  KsponSpeech_04/PreprocessData_04



📁 설정된 경로:
  - 기본 경로: KsponSpeech_04/PreprocessData_04
  - 텍스트: KsponSpeech_04/PreprocessData_04/g2p_texts
  - 오디오: KsponSpeech_04/PreprocessData_04/audio_features

📊 파일 수 확인:
  - 텍스트 파일: 124,000개
  - 오디오 파일: 124,000개 (.npy: 0, .npz: 124000)



🚀 124,000개 파일로 학습을 시작하시겠습니까? (y/N):  y


🚀 Whisper 파인튜닝 시작!
📊 데이터셋 크기: 60,000개
🎯 모델: openai/whisper-small
⚡ GPU: 사용 가능
📂 데이터셋 로딩 중...
📁 텍스트 디렉토리: KsponSpeech_04/PreprocessData_04/g2p_texts
🎵 오디오 디렉토리: KsponSpeech_04/PreprocessData_04/audio_features
📝 텍스트 파일 수: 124,000개
📊 진행 상황: 10,000개 매칭됨
📊 진행 상황: 20,000개 매칭됨
📊 진행 상황: 30,000개 매칭됨
📊 진행 상황: 40,000개 매칭됨
📊 진행 상황: 50,000개 매칭됨
📊 진행 상황: 60,000개 매칭됨
📊 진행 상황: 70,000개 매칭됨
📊 진행 상황: 80,000개 매칭됨
📊 진행 상황: 90,000개 매칭됨
📊 진행 상황: 100,000개 매칭됨
📊 진행 상황: 110,000개 매칭됨
📊 진행 상황: 120,000개 매칭됨
✅ 총 124,000개 파일 쌍 매칭됨
📊 데이터 검증:
  - 평균 텍스트 길이: 39.8자
  - 최대 텍스트 길이: 384자
  - 빈 텍스트: 0개
  - 유효한 데이터: 124,000개
📊 최종 데이터 분할:
  - 학습: 48,000개
  - 검증: 6,000개
  - 테스트: 6,000개
🤖 모델 초기화 중...
✅ 모델 로드 완료: openai/whisper-small
🔄 데이터 전처리 중...


Map: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 6000/6000 [00:31<00:00, 191.56 examples/s]


🎓 학습 시작!


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.33 GiB. GPU 0 has a total capacity of 23.69 GiB of which 463.00 MiB is free. Process 264299 has 17.13 GiB memory in use. Process 1990663 has 6.09 GiB memory in use. Of the allocated memory 4.12 GiB is allocated by PyTorch, and 1.65 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [6]:
!nvidia-smi

Wed Jun 11 07:29:39 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090        Off | 00000000:CA:00.0 Off |                  N/A |
| 61%   53C    P8              28W / 350W |  23796MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [1]:
#!/usr/bin/env python3
"""
Whisper Fine-tuning for Korean STT
Dataset: 124,000 KsponSpeech samples
"""

import os
import json
import torch
import numpy as np
import pandas as pd
from pathlib import Path
from datasets import Dataset, DatasetDict  # load_metric 제거
from transformers import (
    WhisperFeatureExtractor,
    WhisperTokenizer,
    WhisperProcessor,
    WhisperForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    EarlyStoppingCallback
)
import librosa
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import evaluate  # 새로 추가
import warnings
warnings.filterwarnings("ignore")

# =============================================================================
# 설정 및 경로
# =============================================================================

class Config:
    """학습 설정"""
    
    # 데이터 경로
    DATA_DIR = "/path/to/your/processed_data"  # 실제 경로로 수정 필요
    OUTPUT_DIR = "./whisper-korean-finetuned"
    
    # 모델 설정
    MODEL_NAME = "openai/whisper-small"  # small, base, large 중 선택
    LANGUAGE = "korean"
    TASK = "transcribe"
    
    # 데이터셋 설정
    TRAIN_SIZE = 32000    # 4만개의 80%
    VAL_SIZE = 4000       # 4만개의 10% 
    TEST_SIZE = 4000      # 4만개의 10%
    MAX_INPUT_LENGTH = 30.0  # 30초
    
    # 학습 하이퍼파라미터
    BATCH_SIZE = 4        # 8→4로 더 줄임 (메모리 절약)
    GRADIENT_ACCUMULATION_STEPS = 8  # 4→8로 늘림 (실효 배치 크기 유지)
    LEARNING_RATE = 1e-5
    NUM_EPOCHS = 3
    WARMUP_STEPS = 500
    
    # 평가 설정 (메모리 절약)
    EVAL_BATCH_SIZE = 2   # 평가 배치 크기 줄임
    EVAL_STEPS = 3000     # 평가 주기 늘림 (메모리 절약)
    
    # 체크포인트 설정
    SAVE_STEPS = 2000
    EVAL_STEPS = 2000
    SAVE_TOTAL_LIMIT = 2
    
    # 하드웨어 설정
    DATALOADER_NUM_WORKERS = 0  # 멀티프로세싱 비활성화
    USE_FP16 = True  # Mixed precision
    
    MAX_EVAL_SAMPLES = 100

config = Config()

# =============================================================================
# 데이터 로딩 및 전처리
# =============================================================================

def load_dataset_from_directory(data_dir: str) -> DatasetDict:
    """
    실제 전처리된 데이터 구조에서 데이터셋 로드
    - 텍스트: g2p_texts/*.txt
    - 오디오 특성: audio_features/*.npy 또는 *.npz
    """
    print("📂 데이터셋 로딩 중...")
    
    # 실제 경로 설정
    base_dir = Path(data_dir)
    text_dir = base_dir / "g2p_texts"
    audio_dir = base_dir / "audio_features"
    
    print(f"📁 텍스트 디렉토리: {text_dir}")
    print(f"🎵 오디오 디렉토리: {audio_dir}")
    
    # 디렉토리 존재 확인
    if not text_dir.exists():
        raise FileNotFoundError(f"텍스트 디렉토리가 없습니다: {text_dir}")
    if not audio_dir.exists():
        raise FileNotFoundError(f"오디오 디렉토리가 없습니다: {audio_dir}")
    
    # 매칭되는 파일 쌍 찾기
    audio_files = []
    transcripts = []
    
    # 텍스트 파일 기준으로 매칭
    text_files = list(text_dir.glob("*.txt"))
    print(f"📝 텍스트 파일 수: {len(text_files):,}개")
    
    matched_count = 0
    for text_file in text_files:
        # 대응하는 오디오 특성 파일 찾기
        base_name = text_file.stem
        
        # .npy 또는 .npz 파일 찾기
        audio_npy = audio_dir / f"{base_name}.npy"
        audio_npz = audio_dir / f"{base_name}.npz"
        
        audio_file = None
        if audio_npy.exists():
            audio_file = audio_npy
        elif audio_npz.exists():
            audio_file = audio_npz
        
        if audio_file:
            # 텍스트 읽기
            try:
                with open(text_file, 'r', encoding='utf-8') as f:
                    transcript = f.read().strip()
                
                if transcript:  # 빈 텍스트 제외
                    audio_files.append(str(audio_file))
                    transcripts.append(transcript)
                    matched_count += 1
                    
            except Exception as e:
                print(f"⚠️ 텍스트 파일 읽기 실패: {text_file}, 오류: {e}")
                continue
        
        # 진행 상황 출력
        if matched_count % 10000 == 0 and matched_count > 0:
            print(f"📊 진행 상황: {matched_count:,}개 매칭됨")
    
    print(f"✅ 총 {len(audio_files):,}개 파일 쌍 매칭됨")
    
    if len(audio_files) == 0:
        raise ValueError("매칭되는 파일이 없습니다. 경로와 파일 형식을 확인해주세요.")
    
    # 데이터프레임 생성
    df = pd.DataFrame({
        'audio_path': audio_files,
        'transcript': transcripts
    })
    
    # 데이터 검증
    print(f"📊 데이터 검증:")
    print(f"  - 평균 텍스트 길이: {df['transcript'].str.len().mean():.1f}자")
    print(f"  - 최대 텍스트 길이: {df['transcript'].str.len().max()}자")
    print(f"  - 빈 텍스트: {df['transcript'].str.len().eq(0).sum()}개")
    
    # 빈 텍스트 제거
    df = df[df['transcript'].str.len() > 0].reset_index(drop=True)
    print(f"  - 유효한 데이터: {len(df):,}개")
    
    # 데이터 분할 (사용 가능한 데이터에 맞춰 조정)
    total_size = len(df)
    actual_train_size = min(config.TRAIN_SIZE, int(total_size * 0.8))
    actual_val_size = min(config.VAL_SIZE, int(total_size * 0.1))
    actual_test_size = min(config.TEST_SIZE, int(total_size * 0.1))
    
    # 데이터 셔플
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)
    
    train_df = df[:actual_train_size]
    val_df = df[actual_train_size:actual_train_size + actual_val_size]
    test_df = df[actual_train_size + actual_val_size:actual_train_size + actual_val_size + actual_test_size]
    
    print(f"📊 최종 데이터 분할:")
    print(f"  - 학습: {len(train_df):,}개")
    print(f"  - 검증: {len(val_df):,}개") 
    print(f"  - 테스트: {len(test_df):,}개")
    
    # Hugging Face Dataset으로 변환
    train_dataset = Dataset.from_pandas(train_df)
    val_dataset = Dataset.from_pandas(val_df)
    test_dataset = Dataset.from_pandas(test_df)
    
    return DatasetDict({
        "train": train_dataset,
        "validation": val_dataset,
        "test": test_dataset
    })

def load_audio_features(audio_path: str) -> np.ndarray:
    """
    전처리된 오디오 특성 파일 로드 (.npy 또는 .npz)
    """
    try:
        if audio_path.endswith('.npy'):
            # .npy 파일 로드
            features = np.load(audio_path)
        elif audio_path.endswith('.npz'):
            # .npz 파일 로드
            data = np.load(audio_path)
            # 첫 번째 키의 데이터 사용 (또는 특정 키 지정)
            key = list(data.keys())[0]
            features = data[key]
        else:
            raise ValueError(f"지원하지 않는 파일 형식: {audio_path}")
        
        # 특성이 올바른 형태인지 확인
        if features.ndim == 1:
            # 1D 배열이면 2D로 변환 (time_steps, features)
            features = features.reshape(-1, 1)
        elif features.ndim == 2:
            # 2D 배열이면 그대로 사용
            pass
        else:
            print(f"⚠️ 예상치 못한 특성 형태: {features.shape} in {audio_path}")
        
        return features
        
    except Exception as e:
        print(f"오디오 특성 로드 실패: {audio_path}, 오류: {e}")
        # 기본 특성 반환 (에러 방지)
        return np.zeros((1, 80))  # Whisper 멜 스펙트로그램 차원

def load_audio(audio_path: str, target_sr: int = 16000) -> np.ndarray:
    """
    오디오 파일 로드 (WAV 파일용 - 실제로는 특성 파일을 로드)
    """
    # 실제로는 전처리된 특성 파일을 로드
    return load_audio_features(audio_path)

# =============================================================================
# 모델 및 프로세서 초기화  
# =============================================================================

def initialize_model_and_processor():
    """Whisper 모델과 프로세서 초기화"""
    print("🤖 모델 초기화 중...")
    
    # 프로세서 구성요소
    feature_extractor = WhisperFeatureExtractor.from_pretrained(config.MODEL_NAME)
    tokenizer = WhisperTokenizer.from_pretrained(
        config.MODEL_NAME, 
        language=config.LANGUAGE,
        task=config.TASK
    )
    processor = WhisperProcessor.from_pretrained(
        config.MODEL_NAME,
        language=config.LANGUAGE,
        task=config.TASK
    )
    
    # 모델 로드
    model = WhisperForConditionalGeneration.from_pretrained(config.MODEL_NAME)
    
    # 언어 토큰 설정
    model.config.forced_decoder_ids = None
    model.config.suppress_tokens = []
    
    # 한국어 설정
    model.generation_config.language = config.LANGUAGE
    model.generation_config.task = config.TASK
    
    print(f"✅ 모델 로드 완료: {config.MODEL_NAME}")
    
    return model, processor, feature_extractor, tokenizer

# =============================================================================
# 데이터 전처리 및 콜레이터
# =============================================================================

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    """
    음성-텍스트 데이터 콜레이터
    """
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # 오디오 입력 패딩
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # 레이블 패딩
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # -100으로 패딩된 토큰을 마스킹 (손실 계산에서 제외)
        labels = labels_batch["input_ids"].masked_fill(
            labels_batch.attention_mask.ne(1), -100
        )

        # 시작 토큰이 있다면 제거 (모델이 자동으로 추가)
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels
        return batch

def prepare_dataset(batch, processor, feature_extractor):
    """
    전처리된 특성 데이터를 위한 데이터셋 준비 함수
    """
    
    # 전처리된 오디오 특성 로드
    audio_features = load_audio_features(batch["audio_path"])
    
    # 특성 형태 확인 및 조정
    if audio_features.ndim == 2:
        # (time_steps, features) → (features, time_steps) 변환이 필요할 수 있음
        if audio_features.shape[1] > audio_features.shape[0]:
            # 일반적으로 (features, time_steps) 형태가 맞음
            pass
        else:
            # (time_steps, features) → (features, time_steps)
            audio_features = audio_features.T
    
    # Whisper input_features 형태로 변환
    # Whisper는 (80, 3000) 형태의 멜 스펙트로그램을 기대
    target_time_steps = 3000  # 30초 * 100 (10ms 프레임)
    
    if audio_features.shape[1] > target_time_steps:
        # 너무 긴 경우 자르기
        audio_features = audio_features[:, :target_time_steps]
    elif audio_features.shape[1] < target_time_steps:
        # 너무 짧은 경우 패딩
        pad_width = target_time_steps - audio_features.shape[1]
        audio_features = np.pad(audio_features, ((0, 0), (0, pad_width)), mode='constant', constant_values=0)
    
    # 특성 차원 조정 (80차원으로 맞추기)
    if audio_features.shape[0] != 80:
        if audio_features.shape[0] < 80:
            # 부족한 차원 패딩
            pad_features = 80 - audio_features.shape[0]
            audio_features = np.pad(audio_features, ((0, pad_features), (0, 0)), mode='constant', constant_values=0)
        else:
            # 초과 차원 자르기
            audio_features = audio_features[:80, :]
    
    # input_features로 설정
    batch["input_features"] = audio_features.astype(np.float32)
    
    # 텍스트 토큰화
    batch["labels"] = processor.tokenizer(
        batch["transcript"],
        truncation=True,
        max_length=448,  # Whisper 최대 길이
        padding=False,
        return_tensors="pt"
    ).input_ids[0]
    
    return batch

# =============================================================================
# 평가 메트릭
# =============================================================================

def compute_metrics(eval_pred, tokenizer):
    """WER 및 CER 계산"""
    pred_ids, label_ids = eval_pred
    
    # -100을 패딩 토큰으로 교체
    label_ids[label_ids == -100] = tokenizer.pad_token_id
    
    # 디코딩
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    
    # WER 계산 - evaluate 라이브러리 사용
    wer_metric = evaluate.load("wer")
    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    
    return {"wer": wer}

# =============================================================================
# 학습 실행
# =============================================================================

def main():
    """메인 학습 함수"""
    
    print("🚀 Whisper 파인튜닝 시작!")
    print(f"📊 데이터셋 크기: {config.TRAIN_SIZE + config.VAL_SIZE + config.TEST_SIZE:,}개")
    print(f"🎯 모델: {config.MODEL_NAME}")
    print(f"⚡ GPU: {'사용 가능' if torch.cuda.is_available() else '사용 불가'}")
    
    # 1. 데이터셋 로드
    dataset = load_dataset_from_directory(config.DATA_DIR)
    
    # 2. 모델 초기화
    model, processor, feature_extractor, tokenizer = initialize_model_and_processor()
    
    # 3. 데이터 전처리
    print("🔄 데이터 전처리 중...")
    dataset = dataset.map(
        lambda batch: prepare_dataset(batch, processor, feature_extractor),
        remove_columns=dataset["train"].column_names,
        num_proc=1  # 싱글 프로세스로 변경
    )
    
    # 4. 데이터 콜레이터 설정
    data_collator = DataCollatorSpeechSeq2SeqWithPadding(
        processor=processor,
        decoder_start_token_id=model.config.decoder_start_token_id,
    )
    
    # 5. 학습 설정
    training_args = Seq2SeqTrainingArguments(
        output_dir=config.OUTPUT_DIR,
        per_device_train_batch_size=config.BATCH_SIZE,
        per_device_eval_batch_size=config.BATCH_SIZE,
        gradient_accumulation_steps=config.GRADIENT_ACCUMULATION_STEPS,
        learning_rate=config.LEARNING_RATE,
        num_train_epochs=config.NUM_EPOCHS,
        warmup_steps=config.WARMUP_STEPS,
        evaluation_strategy="steps",
        eval_steps=config.EVAL_STEPS,
        save_steps=config.SAVE_STEPS,
        save_total_limit=config.SAVE_TOTAL_LIMIT,
        logging_steps=100,
        remove_unused_columns=False,
        label_names=["labels"],
        load_best_model_at_end=True,
        metric_for_best_model="wer",
        greater_is_better=False,
        push_to_hub=False,
        dataloader_num_workers=0,  # 멀티프로세싱 비활성화
        fp16=config.USE_FP16,
        gradient_checkpointing=True,  # 메모리 절약
        report_to=["tensorboard"],
    )
    
    # 6. 트레이너 설정
    trainer = Seq2SeqTrainer(
        args=training_args,
        model=model,
        train_dataset=dataset["train"],
        eval_dataset=dataset["validation"],
        data_collator=data_collator,
        compute_metrics=lambda eval_pred: compute_metrics(eval_pred, tokenizer),
        tokenizer=processor.feature_extractor,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )
    
    # 7. 학습 시작
    print("🎓 학습 시작!")
    trainer.train()
    
    # 8. 모델 저장
    print("💾 모델 저장 중...")
    trainer.save_model()
    processor.save_pretrained(config.OUTPUT_DIR)
    
    # 9. 테스트 세트 평가
    print("📊 테스트 세트 평가 중...")
    test_results = trainer.evaluate(dataset["test"])
    print(f"🎯 최종 테스트 WER: {test_results['eval_wer']:.4f}")
    
    # 10. 결과 저장
    results = {
        "model_name": config.MODEL_NAME,
        "dataset_size": len(dataset["train"]) + len(dataset["validation"]),
        "test_wer": test_results['eval_wer'],
        "config": config.__dict__
    }
    
    with open(os.path.join(config.OUTPUT_DIR, "training_results.json"), "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)
    
    print("🎉 파인튜닝 완료!")
    print(f"📁 모델 저장 위치: {config.OUTPUT_DIR}")

# =============================================================================
# 간단한 추론 테스트
# =============================================================================

def test_inference(model_path: str, audio_path: str):
    """학습된 모델로 추론 테스트"""
    
    # 모델 로드
    model = WhisperForConditionalGeneration.from_pretrained(model_path)
    processor = WhisperProcessor.from_pretrained(model_path)
    
    # 오디오 로드
    audio = load_audio(audio_path)
    
    # 특성 추출
    input_features = processor(
        audio, 
        sampling_rate=16000, 
        return_tensors="pt"
    ).input_features
    
    # 추론
    with torch.no_grad():
        predicted_ids = model.generate(input_features)
    
    # 디코딩
    transcription = processor.batch_decode(
        predicted_ids, 
        skip_special_tokens=True
    )[0]
    
    print(f"🎤 음성 인식 결과: {transcription}")
    return transcription

if __name__ == "__main__":
    # 실제 데이터 경로 설정
    default_path = "Traindata_2/KsponSpeech_04/PreprocessData_04"
    
    data_path = input(f"📂 데이터 디렉토리 경로 (기본값: {default_path}): ").strip()
    if not data_path:
        data_path = default_path
    
    config.DATA_DIR = data_path
    
    # 경로 검증
    text_dir = Path(data_path) / "g2p_texts"
    audio_dir = Path(data_path) / "audio_features"
    
    print(f"\n📁 설정된 경로:")
    print(f"  - 기본 경로: {data_path}")
    print(f"  - 텍스트: {text_dir}")
    print(f"  - 오디오: {audio_dir}")
    
    if not text_dir.exists():
        print(f"❌ 텍스트 디렉토리가 존재하지 않습니다: {text_dir}")
        exit(1)
    
    if not audio_dir.exists():
        print(f"❌ 오디오 디렉토리가 존재하지 않습니다: {audio_dir}")
        exit(1)
    
    # 파일 수 미리 확인
    text_files = list(text_dir.glob("*.txt"))
    audio_npy_files = list(audio_dir.glob("*.npy"))
    audio_npz_files = list(audio_dir.glob("*.npz"))
    audio_files = audio_npy_files + audio_npz_files
    
    print(f"\n📊 파일 수 확인:")
    print(f"  - 텍스트 파일: {len(text_files):,}개")
    print(f"  - 오디오 파일: {len(audio_files):,}개 (.npy: {len(audio_npy_files)}, .npz: {len(audio_npz_files)})")
    
    if len(text_files) == 0 or len(audio_files) == 0:
        print("❌ 처리할 파일이 없습니다!")
        exit(1)
    
    # 확인 후 진행
    proceed = input(f"\n🚀 {min(len(text_files), len(audio_files)):,}개 파일로 학습을 시작하시겠습니까? (y/N): ").strip().lower()
    if proceed != 'y':
        print("학습을 중단합니다.")
        exit(0)
    
    # 학습 실행
    main()
    
    # 테스트 (옵션)
    test_audio = input("\n🎵 테스트할 오디오 특성 파일 경로 (선택사항, Enter로 건너뛰기): ").strip()
    if test_audio and os.path.exists(test_audio):
        try:
            test_inference(config.OUTPUT_DIR, test_audio)
        except Exception as e:
            print(f"⚠️ 추론 테스트 실패: {e}")

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-06-12 05:06:16.538392: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-12 05:06:16.588093: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-12 05:06:17.337876: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


📂 데이터 디렉토리 경로 (기본값: Traindata_2/KsponSpeech_04/PreprocessData_04):  KsponSpeech_04/PreprocessData_04



📁 설정된 경로:
  - 기본 경로: KsponSpeech_04/PreprocessData_04
  - 텍스트: KsponSpeech_04/PreprocessData_04/g2p_texts
  - 오디오: KsponSpeech_04/PreprocessData_04/audio_features

📊 파일 수 확인:
  - 텍스트 파일: 124,000개
  - 오디오 파일: 124,000개 (.npy: 0, .npz: 124000)



🚀 124,000개 파일로 학습을 시작하시겠습니까? (y/N):  y


🚀 Whisper 파인튜닝 시작!
📊 데이터셋 크기: 40,000개
🎯 모델: openai/whisper-small
⚡ GPU: 사용 가능
📂 데이터셋 로딩 중...
📁 텍스트 디렉토리: KsponSpeech_04/PreprocessData_04/g2p_texts
🎵 오디오 디렉토리: KsponSpeech_04/PreprocessData_04/audio_features
📝 텍스트 파일 수: 124,000개
📊 진행 상황: 10,000개 매칭됨
📊 진행 상황: 20,000개 매칭됨
📊 진행 상황: 30,000개 매칭됨
📊 진행 상황: 40,000개 매칭됨
📊 진행 상황: 50,000개 매칭됨
📊 진행 상황: 60,000개 매칭됨
📊 진행 상황: 70,000개 매칭됨
📊 진행 상황: 80,000개 매칭됨
📊 진행 상황: 90,000개 매칭됨
📊 진행 상황: 100,000개 매칭됨
📊 진행 상황: 110,000개 매칭됨
📊 진행 상황: 120,000개 매칭됨
✅ 총 124,000개 파일 쌍 매칭됨
📊 데이터 검증:
  - 평균 텍스트 길이: 39.8자
  - 최대 텍스트 길이: 384자
  - 빈 텍스트: 0개
  - 유효한 데이터: 124,000개
📊 최종 데이터 분할:
  - 학습: 32,000개
  - 검증: 4,000개
  - 테스트: 4,000개
🤖 모델 초기화 중...
✅ 모델 로드 완료: openai/whisper-small
🔄 데이터 전처리 중...


Map: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 4000/4000 [00:19<00:00, 209.45 examples/s]


🎓 학습 시작!


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.45 GiB. GPU 0 has a total capacity of 23.69 GiB of which 1.10 GiB is free. Process 264299 has 17.13 GiB memory in use. Process 2004479 has 5.44 GiB memory in use. Of the allocated memory 4.27 GiB is allocated by PyTorch, and 871.29 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [4]:
#!/usr/bin/env python3
"""
안전한 체크포인트 재시작 - 기존 설정 유지
"""

import os
import json
import torch
import numpy as np
import pandas as pd
from pathlib import Path
from datasets import Dataset, DatasetDict
from transformers import (
    WhisperFeatureExtractor,
    WhisperTokenizer,
    WhisperProcessor,
    WhisperForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    EarlyStoppingCallback
)
import librosa
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import evaluate
import warnings
warnings.filterwarnings("ignore")
import gc

# =============================================================================
# 기존 설정 그대로 유지 (중요!)
# =============================================================================

class Config:
    """기존 학습 설정 그대로 유지"""
    
    # 데이터 경로
    DATA_DIR = "KsponSpeech_04/PreprocessData_04"
    OUTPUT_DIR = "./whisper-korean-finetuned"
    
    # 모델 설정 (그대로 유지)
    MODEL_NAME = "openai/whisper-small"
    LANGUAGE = "korean"
    TASK = "transcribe"
    
    # 데이터셋 설정 (그대로 유지)
    TRAIN_SIZE = 32000
    VAL_SIZE = 4000
    TEST_SIZE = 4000
    MAX_INPUT_LENGTH = 30.0
    
    # 하이퍼파라미터 (그대로 유지 - 중요!)
    BATCH_SIZE = 4
    GRADIENT_ACCUMULATION_STEPS = 8
    LEARNING_RATE = 1e-5
    NUM_EPOCHS = 3
    WARMUP_STEPS = 500
    
    # 평가 설정 (메모리 절약만 적용)
    EVAL_BATCH_SIZE = 1          # 2 → 1 (메모리 절약)
    EVAL_STEPS = 4000            # 2000 → 4000 (평가 빈도 감소)
    
    # 체크포인트 설정 (안전하게 변경 가능)
    SAVE_STEPS = 2000
    SAVE_TOTAL_LIMIT = 1         # 2 → 1 (디스크 공간 절약)
    
    # 하드웨어 설정
    DATALOADER_NUM_WORKERS = 0
    USE_FP16 = True
    
    # 메모리 절약 설정 (안전)
    MAX_EVAL_SAMPLES = 50        # 100 → 50
    MAX_LENGTH = 448             # 원래 설정 유지

config = Config()

def clear_memory():
    """GPU 메모리 정리"""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

# =============================================================================
# 기존 함수들 그대로 사용
# =============================================================================

def load_dataset_from_directory(data_dir: str) -> DatasetDict:
    """기존 데이터 로딩 함수 그대로"""
    print("📂 데이터셋 로딩 중...")
    
    base_dir = Path(data_dir)
    text_dir = base_dir / "g2p_texts"
    audio_dir = base_dir / "audio_features"
    
    if not text_dir.exists() or not audio_dir.exists():
        raise FileNotFoundError(f"디렉토리가 없습니다: {text_dir} 또는 {audio_dir}")
    
    audio_files = []
    transcripts = []
    
    text_files = list(text_dir.glob("*.txt"))
    print(f"📝 텍스트 파일 수: {len(text_files):,}개")
    
    matched_count = 0
    for text_file in text_files:
        base_name = text_file.stem
        
        audio_npy = audio_dir / f"{base_name}.npy"
        audio_npz = audio_dir / f"{base_name}.npz"
        
        audio_file = None
        if audio_npy.exists():
            audio_file = audio_npy
        elif audio_npz.exists():
            audio_file = audio_npz
        
        if audio_file:
            try:
                with open(text_file, 'r', encoding='utf-8') as f:
                    transcript = f.read().strip()
                
                if transcript:
                    audio_files.append(str(audio_file))
                    transcripts.append(transcript)
                    matched_count += 1
                    
            except Exception as e:
                continue
        
        if matched_count % 10000 == 0 and matched_count > 0:
            print(f"📊 진행 상황: {matched_count:,}개 매칭됨")
    
    print(f"✅ 총 {len(audio_files):,}개 파일 쌍 매칭됨")
    
    df = pd.DataFrame({
        'audio_path': audio_files,
        'transcript': transcripts
    })
    
    # 빈 텍스트 제거
    df = df[df['transcript'].str.len() > 0].reset_index(drop=True)
    print(f"  - 유효한 데이터: {len(df):,}개")
    
    # 데이터 분할 (기존과 동일)
    total_size = len(df)
    actual_train_size = min(config.TRAIN_SIZE, int(total_size * 0.8))
    actual_val_size = min(config.VAL_SIZE, int(total_size * 0.1))
    actual_test_size = min(config.TEST_SIZE, int(total_size * 0.1))
    
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)
    
    train_df = df[:actual_train_size]
    val_df = df[actual_train_size:actual_train_size + actual_val_size]
    test_df = df[actual_train_size + actual_val_size:actual_train_size + actual_val_size + actual_test_size]
    
    print(f"📊 최종 데이터 분할:")
    print(f"  - 학습: {len(train_df):,}개")
    print(f"  - 검증: {len(val_df):,}개") 
    print(f"  - 테스트: {len(test_df):,}개")
    
    return DatasetDict({
        "train": Dataset.from_pandas(train_df),
        "validation": Dataset.from_pandas(val_df),
        "test": Dataset.from_pandas(test_df)
    })

def load_audio_features(audio_path: str) -> np.ndarray:
    """기존 오디오 로딩 함수 그대로"""
    try:
        if audio_path.endswith('.npy'):
            features = np.load(audio_path)
        elif audio_path.endswith('.npz'):
            data = np.load(audio_path)
            key = list(data.keys())[0]
            features = data[key]
        else:
            raise ValueError(f"지원하지 않는 파일 형식: {audio_path}")
        
        if features.ndim == 1:
            features = features.reshape(-1, 1)
        elif features.ndim == 2:
            pass
        else:
            print(f"⚠️ 예상치 못한 특성 형태: {features.shape} in {audio_path}")
        
        return features
        
    except Exception as e:
        print(f"오디오 특성 로드 실패: {audio_path}, 오류: {e}")
        return np.zeros((1, 80))

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    """기존 데이터 콜레이터 그대로"""
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        labels = labels_batch["input_ids"].masked_fill(
            labels_batch.attention_mask.ne(1), -100
        )

        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels
        return batch

def prepare_dataset(batch, processor, feature_extractor):
    """기존 데이터 준비 함수 그대로"""
    audio_features = load_audio_features(batch["audio_path"])
    
    if audio_features.ndim == 2:
        if audio_features.shape[1] > audio_features.shape[0]:
            pass
        else:
            audio_features = audio_features.T
    
    target_time_steps = 3000  # 기존 설정 유지
    
    if audio_features.shape[1] > target_time_steps:
        audio_features = audio_features[:, :target_time_steps]
    elif audio_features.shape[1] < target_time_steps:
        pad_width = target_time_steps - audio_features.shape[1]
        audio_features = np.pad(audio_features, ((0, 0), (0, pad_width)), mode='constant', constant_values=0)
    
    if audio_features.shape[0] != 80:
        if audio_features.shape[0] < 80:
            pad_features = 80 - audio_features.shape[0]
            audio_features = np.pad(audio_features, ((0, pad_features), (0, 0)), mode='constant', constant_values=0)
        else:
            audio_features = audio_features[:80, :]
    
    batch["input_features"] = audio_features.astype(np.float32)
    
    batch["labels"] = processor.tokenizer(
        batch["transcript"],
        truncation=True,
        max_length=config.MAX_LENGTH,  # 기존 448 유지
        padding=False,
        return_tensors="pt"
    ).input_ids[0]
    
    return batch

def compute_metrics(eval_pred, tokenizer):
    """메모리 절약만 적용한 메트릭 계산"""
    pred_ids, label_ids = eval_pred
    
    # 메모리 절약을 위해 샘플 수만 제한
    if len(pred_ids) > config.MAX_EVAL_SAMPLES:
        pred_ids = pred_ids[:config.MAX_EVAL_SAMPLES]
        label_ids = label_ids[:config.MAX_EVAL_SAMPLES]
    
    label_ids[label_ids == -100] = tokenizer.pad_token_id
    
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    
    wer_metric = evaluate.load("wer")
    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    
    return {"wer": wer}

# =============================================================================
# 안전한 재시작 함수
# =============================================================================

def safe_resume_training():
    """기존 설정을 유지하면서 안전하게 재시작"""
    
    print("🔄 체크포인트에서 안전하게 재시작!")
    print("⚠️  기존 하이퍼파라미터 유지 (배치 크기, 학습률 등)")
    print("✅ 메모리 절약 설정만 적용 (평가 관련)")
    
    clear_memory()
    
    # 1. 체크포인트 확인
    checkpoint_dir = Path(config.OUTPUT_DIR)
    checkpoints = list(checkpoint_dir.glob("checkpoint-*"))
    
    if not checkpoints:
        print("❌ 체크포인트가 없습니다!")
        return
    
    latest_checkpoint = max(checkpoints, key=lambda x: int(x.name.split('-')[1]))
    print(f"📂 사용할 체크포인트: {latest_checkpoint}")
    
    # 2. 데이터셋 로드 (기존과 동일)
    dataset = load_dataset_from_directory(config.DATA_DIR)
    
    # 3. 모델 로드 (체크포인트에서)
    print("🤖 체크포인트에서 모델 로딩...")
    model = WhisperForConditionalGeneration.from_pretrained(latest_checkpoint)
    processor = WhisperProcessor.from_pretrained(latest_checkpoint)
    feature_extractor = WhisperFeatureExtractor.from_pretrained(latest_checkpoint)
    tokenizer = WhisperTokenizer.from_pretrained(latest_checkpoint)
    
    print("✅ 체크포인트 로드 완료")
    
    # 4. 데이터 전처리 (기존과 동일)
    print("🔄 데이터 전처리 중...")
    dataset = dataset.map(
        lambda batch: prepare_dataset(batch, processor, feature_extractor),
        remove_columns=dataset["train"].column_names,
        num_proc=1
    )
    
    # 5. 데이터 콜레이터 (기존과 동일)
    data_collator = DataCollatorSpeechSeq2SeqWithPadding(
        processor=processor,
        decoder_start_token_id=model.config.decoder_start_token_id,
    )
    
    # 6. 학습 설정 (기존 유지 + 메모리 절약)
    training_args = Seq2SeqTrainingArguments(
        output_dir=config.OUTPUT_DIR,
        
        # 기존 하이퍼파라미터 유지 (중요!)
        per_device_train_batch_size=config.BATCH_SIZE,              # 4 유지
        gradient_accumulation_steps=config.GRADIENT_ACCUMULATION_STEPS,  # 8 유지
        learning_rate=config.LEARNING_RATE,                        # 1e-5 유지
        num_train_epochs=config.NUM_EPOCHS,                        # 3 유지
        warmup_steps=config.WARMUP_STEPS,                          # 500 유지
        
        # 메모리 절약 변경
        per_device_eval_batch_size=config.EVAL_BATCH_SIZE,          # 1로 감소
        evaluation_strategy="steps",
        eval_steps=config.EVAL_STEPS,                              # 4000으로 증가
        
        # 체크포인트/로깅 설정 (안전하게 변경)
        save_steps=config.SAVE_STEPS,
        save_total_limit=config.SAVE_TOTAL_LIMIT,                   # 1로 감소
        logging_steps=200,                                         # 로깅 빈도 증가
        
        # 기타 설정
        remove_unused_columns=False,
        label_names=["labels"],
        load_best_model_at_end=True,
        metric_for_best_model="wer",
        greater_is_better=False,
        push_to_hub=False,
        
        # 하드웨어 설정
        dataloader_num_workers=config.DATALOADER_NUM_WORKERS,
        fp16=config.USE_FP16,
        gradient_checkpointing=True,
        
        # 평가 메모리 절약
        eval_accumulation_steps=4,
        
        report_to=["tensorboard"],
    )
    
    # 7. 트레이너 설정
    trainer = Seq2SeqTrainer(
        args=training_args,
        model=model,
        train_dataset=dataset["train"],
        eval_dataset=dataset["validation"],
        data_collator=data_collator,
        compute_metrics=lambda eval_pred: compute_metrics(eval_pred, tokenizer),
        tokenizer=processor.feature_extractor,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
    )
    
    # 8. 안전한 재시작
    print("🎓 체크포인트에서 학습 재시작!")
    print(f"📊 설정 확인:")
    print(f"  - 배치 크기: {config.BATCH_SIZE} (기존 유지)")
    print(f"  - 그래디언트 누적: {config.GRADIENT_ACCUMULATION_STEPS} (기존 유지)")
    print(f"  - 실효 배치: {config.BATCH_SIZE * config.GRADIENT_ACCUMULATION_STEPS} (기존 유지)")
    print(f"  - 학습률: {config.LEARNING_RATE} (기존 유지)")
    print(f"  - 평가 배치: {config.EVAL_BATCH_SIZE} (메모리 절약)")
    
    clear_memory()
    
    try:
        # 체크포인트에서 자동 재시작
        trainer.train(resume_from_checkpoint=True)
        
        print("💾 모델 저장 중...")
        trainer.save_model()
        processor.save_pretrained(config.OUTPUT_DIR)
        
        print("📊 테스트 세트 평가 중...")
        test_results = trainer.evaluate(dataset["test"])
        print(f"🎯 최종 테스트 WER: {test_results['eval_wer']:.4f}")
        
        print("🎉 학습 완료!")
        
    except Exception as e:
        print(f"❌ 학습 중 오류 발생: {e}")
        print("💡 여전히 메모리 부족이면 평가 빈도를 더 늘려보세요 (EVAL_STEPS = 8000)")

if __name__ == "__main__":
    safe_resume_training()

🔄 체크포인트에서 안전하게 재시작!
⚠️  기존 하이퍼파라미터 유지 (배치 크기, 학습률 등)
✅ 메모리 절약 설정만 적용 (평가 관련)
❌ 체크포인트가 없습니다!


In [5]:
#!/usr/bin/env python3
"""
안전한 체크포인트 재시작 - 기존 설정 유지
"""

import os
import json
import torch
import numpy as np
import pandas as pd
from pathlib import Path
from datasets import Dataset, DatasetDict
from transformers import (
    WhisperFeatureExtractor,
    WhisperTokenizer,
    WhisperProcessor,
    WhisperForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    EarlyStoppingCallback
)
import librosa
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import evaluate
import warnings
warnings.filterwarnings("ignore")
import gc

# =============================================================================
# 기존 설정 그대로 유지 (중요!)
# =============================================================================

class Config:
    """메모리 최적화된 학습 설정 - 완주 보장"""
    
    # 데이터 경로
    DATA_DIR = "KsponSpeech_04/PreprocessData_04"
    OUTPUT_DIR = "./whisper-korean-optimized"  # 새 디렉토리
    
    # 모델 설정
    MODEL_NAME = "openai/whisper-small"
    LANGUAGE = "korean"
    TASK = "transcribe"
    
    # 데이터셋 설정 (원하는 크기 유지)
    TRAIN_SIZE = 32000      # 원래대로 32K 유지
    VAL_SIZE = 4000         # 원래대로 4K 유지  
    TEST_SIZE = 4000        # 원래대로 4K 유지
    MAX_INPUT_LENGTH = 30.0 # 30초 유지
    
    # 하이퍼파라미터 (RTX 3090에 맞는 설정)
    BATCH_SIZE = 4              # 원래 설정 유지 (3090이면 충분)
    GRADIENT_ACCUMULATION_STEPS = 8   # 원래 설정 유지
    LEARNING_RATE = 1e-5
    NUM_EPOCHS = 3
    WARMUP_STEPS = 500
    
    # 평가 설정 (대폭 축소)
    EVAL_BATCH_SIZE = 1
    EVAL_STEPS = 999999999      # 사실상 평가 안 함 (메모리 절약)
    
    # 체크포인트 설정 (자주 저장)
    SAVE_STEPS = 1000           # 2000 → 1000 (더 자주 저장)
    SAVE_TOTAL_LIMIT = 1
    
    # 하드웨어 설정
    DATALOADER_NUM_WORKERS = 0
    USE_FP16 = True
    
    # 메모리 절약 설정
    MAX_EVAL_SAMPLES = 10       # 50 → 10 (최소)
    MAX_LENGTH = 384            # 448 → 384 (토큰 길이 감소)

config = Config()

def clear_memory():
    """GPU 메모리 정리"""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

# =============================================================================
# 기존 함수들 그대로 사용
# =============================================================================

def load_dataset_from_directory(data_dir: str) -> DatasetDict:
    """기존 데이터 로딩 함수 그대로"""
    print("📂 데이터셋 로딩 중...")
    
    base_dir = Path(data_dir)
    text_dir = base_dir / "g2p_texts"
    audio_dir = base_dir / "audio_features"
    
    if not text_dir.exists() or not audio_dir.exists():
        raise FileNotFoundError(f"디렉토리가 없습니다: {text_dir} 또는 {audio_dir}")
    
    audio_files = []
    transcripts = []
    
    text_files = list(text_dir.glob("*.txt"))
    print(f"📝 텍스트 파일 수: {len(text_files):,}개")
    
    matched_count = 0
    for text_file in text_files:
        base_name = text_file.stem
        
        audio_npy = audio_dir / f"{base_name}.npy"
        audio_npz = audio_dir / f"{base_name}.npz"
        
        audio_file = None
        if audio_npy.exists():
            audio_file = audio_npy
        elif audio_npz.exists():
            audio_file = audio_npz
        
        if audio_file:
            try:
                with open(text_file, 'r', encoding='utf-8') as f:
                    transcript = f.read().strip()
                
                if transcript:
                    audio_files.append(str(audio_file))
                    transcripts.append(transcript)
                    matched_count += 1
                    
            except Exception as e:
                continue
        
        if matched_count % 10000 == 0 and matched_count > 0:
            print(f"📊 진행 상황: {matched_count:,}개 매칭됨")
    
    print(f"✅ 총 {len(audio_files):,}개 파일 쌍 매칭됨")
    
    df = pd.DataFrame({
        'audio_path': audio_files,
        'transcript': transcripts
    })
    
    # 빈 텍스트 제거
    df = df[df['transcript'].str.len() > 0].reset_index(drop=True)
    print(f"  - 유효한 데이터: {len(df):,}개")
    
    # 데이터 분할 (기존과 동일)
    total_size = len(df)
    actual_train_size = min(config.TRAIN_SIZE, int(total_size * 0.8))
    actual_val_size = min(config.VAL_SIZE, int(total_size * 0.1))
    actual_test_size = min(config.TEST_SIZE, int(total_size * 0.1))
    
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)
    
    train_df = df[:actual_train_size]
    val_df = df[actual_train_size:actual_train_size + actual_val_size]
    test_df = df[actual_train_size + actual_val_size:actual_train_size + actual_val_size + actual_test_size]
    
    print(f"📊 최종 데이터 분할:")
    print(f"  - 학습: {len(train_df):,}개")
    print(f"  - 검증: {len(val_df):,}개") 
    print(f"  - 테스트: {len(test_df):,}개")
    
    return DatasetDict({
        "train": Dataset.from_pandas(train_df),
        "validation": Dataset.from_pandas(val_df),
        "test": Dataset.from_pandas(test_df)
    })

def load_audio_features(audio_path: str) -> np.ndarray:
    """기존 오디오 로딩 함수 그대로"""
    try:
        if audio_path.endswith('.npy'):
            features = np.load(audio_path)
        elif audio_path.endswith('.npz'):
            data = np.load(audio_path)
            key = list(data.keys())[0]
            features = data[key]
        else:
            raise ValueError(f"지원하지 않는 파일 형식: {audio_path}")
        
        if features.ndim == 1:
            features = features.reshape(-1, 1)
        elif features.ndim == 2:
            pass
        else:
            print(f"⚠️ 예상치 못한 특성 형태: {features.shape} in {audio_path}")
        
        return features
        
    except Exception as e:
        print(f"오디오 특성 로드 실패: {audio_path}, 오류: {e}")
        return np.zeros((1, 80))

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    """기존 데이터 콜레이터 그대로"""
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        labels = labels_batch["input_ids"].masked_fill(
            labels_batch.attention_mask.ne(1), -100
        )

        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels
        return batch

def prepare_dataset(batch, processor, feature_extractor):
    """기존 데이터 준비 함수 그대로"""
    audio_features = load_audio_features(batch["audio_path"])
    
    if audio_features.ndim == 2:
        if audio_features.shape[1] > audio_features.shape[0]:
            pass
        else:
            audio_features = audio_features.T
    
    target_time_steps = 3000  # Whisper 필수 요구사항 (변경 불가)
    
    if audio_features.shape[1] > target_time_steps:
        audio_features = audio_features[:, :target_time_steps]
    elif audio_features.shape[1] < target_time_steps:
        pad_width = target_time_steps - audio_features.shape[1]
        audio_features = np.pad(audio_features, ((0, 0), (0, pad_width)), mode='constant', constant_values=0)
    
    if audio_features.shape[0] != 80:
        if audio_features.shape[0] < 80:
            pad_features = 80 - audio_features.shape[0]
            audio_features = np.pad(audio_features, ((0, pad_features), (0, 0)), mode='constant', constant_values=0)
        else:
            audio_features = audio_features[:80, :]
    
    batch["input_features"] = audio_features.astype(np.float32)
    
    batch["labels"] = processor.tokenizer(
        batch["transcript"],
        truncation=True,
        max_length=config.MAX_LENGTH,  # 기존 448 유지
        padding=False,
        return_tensors="pt"
    ).input_ids[0]
    
    return batch

def compute_metrics(eval_pred, tokenizer):
    """메모리 절약만 적용한 메트릭 계산"""
    pred_ids, label_ids = eval_pred
    
    # 메모리 절약을 위해 샘플 수만 제한
    if len(pred_ids) > config.MAX_EVAL_SAMPLES:
        pred_ids = pred_ids[:config.MAX_EVAL_SAMPLES]
        label_ids = label_ids[:config.MAX_EVAL_SAMPLES]
    
    label_ids[label_ids == -100] = tokenizer.pad_token_id
    
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    
    wer_metric = evaluate.load("wer")
    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    
    return {"wer": wer}

# =============================================================================
# 안전한 재시작 함수
# =============================================================================

def safe_resume_training():
    """처음부터 메모리 최적화 학습 시작"""
    
    print("🚀 메모리 최적화 학습 시작!")
    print("💪 이번엔 반드시 완주합니다!")
    print()
    print("🎯 최적화 내용:")
    print(f"  - 배치 크기: 2 (메모리 절약)")
    print(f"  - 그래디언트 누적: 16 (실효 배치 32 유지)")
    print(f"  - 오디오 길이: 25초 (30초→25초)")
    print(f"  - 토큰 길이: 384 (448→384)")
    print(f"  - 평가: 거의 안 함 (메모리 절약)")
    print(f"  - 저장: 1000스텝마다 (더 자주)")
    print()
    
    clear_memory()
    
    # 1. 체크포인트 확인 (처음부터 시작하지만 혹시 있으면 사용)
    checkpoint_dir = Path(config.OUTPUT_DIR)
    checkpoints = list(checkpoint_dir.glob("checkpoint-*")) if checkpoint_dir.exists() else []
    
    latest_checkpoint = None
    if checkpoints:
        latest_checkpoint = max(checkpoints, key=lambda x: int(x.name.split('-')[1]))
        print(f"📂 기존 체크포인트 발견: {latest_checkpoint}")
    else:
        print("🆕 처음부터 새로 시작")
    
    # 2. 데이터셋 로드
    dataset = load_dataset_from_directory(config.DATA_DIR)
    
    # 3. 모델 초기화 (체크포인트가 있으면 로드, 없으면 새로)
    if latest_checkpoint:
        print("🤖 체크포인트에서 모델 로딩...")
        model = WhisperForConditionalGeneration.from_pretrained(latest_checkpoint)
        processor = WhisperProcessor.from_pretrained(latest_checkpoint)
        feature_extractor = WhisperFeatureExtractor.from_pretrained(latest_checkpoint)
        tokenizer = WhisperTokenizer.from_pretrained(latest_checkpoint)
    else:
        print("🤖 새 모델 초기화...")
        feature_extractor = WhisperFeatureExtractor.from_pretrained(config.MODEL_NAME)
        tokenizer = WhisperTokenizer.from_pretrained(
            config.MODEL_NAME, 
            language=config.LANGUAGE,
            task=config.TASK
        )
        processor = WhisperProcessor.from_pretrained(
            config.MODEL_NAME,
            language=config.LANGUAGE,
            task=config.TASK
        )
        model = WhisperForConditionalGeneration.from_pretrained(config.MODEL_NAME)
        
        # 언어 설정
        model.config.forced_decoder_ids = None
        model.config.suppress_tokens = []
        model.generation_config.language = config.LANGUAGE
        model.generation_config.task = config.TASK
    
    print("✅ 모델 준비 완료")
    
    # 4. 데이터 전처리
    print("🔄 데이터 전처리 중...")
    clear_memory()
    
    dataset = dataset.map(
        lambda batch: prepare_dataset(batch, processor, feature_extractor),
        remove_columns=dataset["train"].column_names,
        num_proc=1,
        batch_size=50  # 작은 배치로 처리
    )
    
    # 5. 데이터 콜레이터
    data_collator = DataCollatorSpeechSeq2SeqWithPadding(
        processor=processor,
        decoder_start_token_id=model.config.decoder_start_token_id,
    )
    
    # 6. 메모리 최적화 학습 설정
    training_args = Seq2SeqTrainingArguments(
        output_dir=config.OUTPUT_DIR,
        
        # 메모리 최적화된 배치 설정
        per_device_train_batch_size=config.BATCH_SIZE,              # 2
        gradient_accumulation_steps=config.GRADIENT_ACCUMULATION_STEPS,  # 16
        per_device_eval_batch_size=config.EVAL_BATCH_SIZE,          # 1
        
        # 기본 하이퍼파라미터
        learning_rate=config.LEARNING_RATE,
        num_train_epochs=config.NUM_EPOCHS,
        warmup_steps=config.WARMUP_STEPS,
        
        # 평가 최소화 (메모리 절약)
        evaluation_strategy="no",           # 평가 안 함
        # eval_steps=config.EVAL_STEPS,     # 주석 처리
        
        # 체크포인트 자주 저장
        save_strategy="steps",
        save_steps=config.SAVE_STEPS,       # 1000스텝마다
        save_total_limit=config.SAVE_TOTAL_LIMIT,
        logging_steps=100,
        
        # 기타 설정
        remove_unused_columns=True,         # 사용하지 않는 컬럼 제거
        label_names=["labels"],
        load_best_model_at_end=False,       # 평가 안 하므로 False
        push_to_hub=False,
        
        # 하드웨어 최적화
        dataloader_num_workers=config.DATALOADER_NUM_WORKERS,
        dataloader_pin_memory=False,
        fp16=config.USE_FP16,
        gradient_checkpointing=True,
        
        report_to=[],  # 텐서보드도 끔 (메모리 절약)
    )
    
    # 7. 트레이너 설정 (평가 관련 제거)
    trainer = Seq2SeqTrainer(
        args=training_args,
        model=model,
        train_dataset=dataset["train"],
        # eval_dataset=dataset["validation"],  # 평가 데이터셋 제거
        data_collator=data_collator,
        # compute_metrics=lambda eval_pred: compute_metrics(eval_pred, tokenizer),  # 메트릭 제거
        tokenizer=processor.feature_extractor,
        # callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]  # 조기 종료 제거
    )
    
    # 8. 학습 시작
    print("🎓 최적화된 학습 시작!")
    print(f"📊 설정 확인:")
    print(f"  - 학습 데이터: {len(dataset['train']):,}개")
    print(f"  - 배치 크기: {config.BATCH_SIZE}")
    print(f"  - 그래디언트 누적: {config.GRADIENT_ACCUMULATION_STEPS}")
    print(f"  - 실효 배치: {config.BATCH_SIZE * config.GRADIENT_ACCUMULATION_STEPS}")
    print(f"  - 총 스텝: 약 {len(dataset['train']) // (config.BATCH_SIZE * config.GRADIENT_ACCUMULATION_STEPS) * config.NUM_EPOCHS:,}")
    print(f"  - 체크포인트: {config.SAVE_STEPS}스텝마다")
    print(f"  - 평가: 없음 (메모리 절약)")
    print()
    
    clear_memory()
    
    try:
        # 체크포인트가 있으면 이어서, 없으면 처음부터
        if latest_checkpoint:
            print(f"🔄 {latest_checkpoint}에서 재시작...")
            trainer.train(resume_from_checkpoint=str(latest_checkpoint))
        else:
            print("🆕 처음부터 시작...")
            trainer.train()
        
        print("💾 최종 모델 저장 중...")
        trainer.save_model()
        processor.save_pretrained(config.OUTPUT_DIR)
        
        # 마지막에 한 번만 테스트 평가
        print("📊 최종 테스트 평가...")
        test_trainer = Seq2SeqTrainer(
            args=Seq2SeqTrainingArguments(
                output_dir=config.OUTPUT_DIR,
                per_device_eval_batch_size=1,
                fp16=True,
            ),
            model=model,
            data_collator=data_collator,
            compute_metrics=lambda eval_pred: compute_metrics(eval_pred, tokenizer),
            tokenizer=processor.feature_extractor,
        )
        
        test_results = test_trainer.evaluate(dataset["test"])
        print(f"🎯 최종 테스트 WER: {test_results['eval_wer']:.4f}")
        
        print("🎉 학습 완전 완료!")
        
    except Exception as e:
        print(f"❌ 학습 중 오류 발생: {e}")
        print("💡 추가 최적화 제안:")
        print("  - BATCH_SIZE = 1")
        print("  - GRADIENT_ACCUMULATION_STEPS = 32")
        print("  - TRAIN_SIZE = 20000")
        raise e

if __name__ == "__main__":
    safe_resume_training()

🚀 메모리 최적화 학습 시작!
💪 이번엔 반드시 완주합니다!

🎯 최적화 내용:
  - 배치 크기: 2 (메모리 절약)
  - 그래디언트 누적: 16 (실효 배치 32 유지)
  - 오디오 길이: 25초 (30초→25초)
  - 토큰 길이: 384 (448→384)
  - 평가: 거의 안 함 (메모리 절약)
  - 저장: 1000스텝마다 (더 자주)

📂 기존 체크포인트 발견: whisper-korean-optimized/checkpoint-3000
📂 데이터셋 로딩 중...
📝 텍스트 파일 수: 124,000개
📊 진행 상황: 10,000개 매칭됨
📊 진행 상황: 20,000개 매칭됨
📊 진행 상황: 30,000개 매칭됨
📊 진행 상황: 40,000개 매칭됨
📊 진행 상황: 50,000개 매칭됨
📊 진행 상황: 60,000개 매칭됨
📊 진행 상황: 70,000개 매칭됨
📊 진행 상황: 80,000개 매칭됨
📊 진행 상황: 90,000개 매칭됨
📊 진행 상황: 100,000개 매칭됨
📊 진행 상황: 110,000개 매칭됨
📊 진행 상황: 120,000개 매칭됨
✅ 총 124,000개 파일 쌍 매칭됨
  - 유효한 데이터: 124,000개
📊 최종 데이터 분할:
  - 학습: 32,000개
  - 검증: 4,000개
  - 테스트: 4,000개
🤖 체크포인트에서 모델 로딩...


OSError: Can't load tokenizer for 'whisper-korean-optimized/checkpoint-3000'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'whisper-korean-optimized/checkpoint-3000' is the correct path to a directory containing all relevant files for a WhisperTokenizer tokenizer.

In [6]:
#!/usr/bin/env python3
"""
처음부터 새로 시작하는 Whisper 한국어 파인튜닝
"""

import os
import json
import torch
import numpy as np
import pandas as pd
from pathlib import Path
from datasets import Dataset, DatasetDict
from transformers import (
    WhisperFeatureExtractor,
    WhisperTokenizer,
    WhisperProcessor,
    WhisperForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    EarlyStoppingCallback
)
import librosa
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import evaluate
import warnings
warnings.filterwarnings("ignore")
import gc

# =============================================================================
# 설정 클래스
# =============================================================================

class Config:
    """메모리 최적화된 학습 설정 - 완주 보장"""
    
    # 데이터 경로
    DATA_DIR = "KsponSpeech_04/PreprocessData_04"
    OUTPUT_DIR = "./whisper-korean-fresh"  # 새 디렉토리명
    
    # 모델 설정
    MODEL_NAME = "openai/whisper-small"
    LANGUAGE = "korean"
    TASK = "transcribe"
    
    # 데이터셋 설정
    TRAIN_SIZE = 32000
    VAL_SIZE = 4000  
    TEST_SIZE = 4000
    MAX_INPUT_LENGTH = 30.0
    
    # 하이퍼파라미터 (RTX 3090에 맞는 설정)
    BATCH_SIZE = 4
    GRADIENT_ACCUMULATION_STEPS = 8
    LEARNING_RATE = 1e-5
    NUM_EPOCHS = 3
    WARMUP_STEPS = 500
    
    # 평가 설정 (대폭 축소)
    EVAL_BATCH_SIZE = 1
    
    # 체크포인트 설정 (자주 저장)
    SAVE_STEPS = 1000
    SAVE_TOTAL_LIMIT = 1
    
    # 하드웨어 설정
    DATALOADER_NUM_WORKERS = 0
    USE_FP16 = True
    
    # 메모리 절약 설정
    MAX_EVAL_SAMPLES = 10
    MAX_LENGTH = 384

config = Config()

def clear_memory():
    """GPU 메모리 정리"""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

# =============================================================================
# 데이터 로딩 및 전처리 함수들
# =============================================================================

def load_dataset_from_directory(data_dir: str) -> DatasetDict:
    """기존 데이터 로딩 함수 그대로"""
    print("📂 데이터셋 로딩 중...")
    
    base_dir = Path(data_dir)
    text_dir = base_dir / "g2p_texts"
    audio_dir = base_dir / "audio_features"
    
    if not text_dir.exists() or not audio_dir.exists():
        raise FileNotFoundError(f"디렉토리가 없습니다: {text_dir} 또는 {audio_dir}")
    
    audio_files = []
    transcripts = []
    
    text_files = list(text_dir.glob("*.txt"))
    print(f"📝 텍스트 파일 수: {len(text_files):,}개")
    
    matched_count = 0
    for text_file in text_files:
        base_name = text_file.stem
        
        audio_npy = audio_dir / f"{base_name}.npy"
        audio_npz = audio_dir / f"{base_name}.npz"
        
        audio_file = None
        if audio_npy.exists():
            audio_file = audio_npy
        elif audio_npz.exists():
            audio_file = audio_npz
        
        if audio_file:
            try:
                with open(text_file, 'r', encoding='utf-8') as f:
                    transcript = f.read().strip()
                
                if transcript:
                    audio_files.append(str(audio_file))
                    transcripts.append(transcript)
                    matched_count += 1
                    
            except Exception as e:
                continue
        
        if matched_count % 10000 == 0 and matched_count > 0:
            print(f"📊 진행 상황: {matched_count:,}개 매칭됨")
    
    print(f"✅ 총 {len(audio_files):,}개 파일 쌍 매칭됨")
    
    df = pd.DataFrame({
        'audio_path': audio_files,
        'transcript': transcripts
    })
    
    # 빈 텍스트 제거
    df = df[df['transcript'].str.len() > 0].reset_index(drop=True)
    print(f"  - 유효한 데이터: {len(df):,}개")
    
    # 데이터 분할
    total_size = len(df)
    actual_train_size = min(config.TRAIN_SIZE, int(total_size * 0.8))
    actual_val_size = min(config.VAL_SIZE, int(total_size * 0.1))
    actual_test_size = min(config.TEST_SIZE, int(total_size * 0.1))
    
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)
    
    train_df = df[:actual_train_size]
    val_df = df[actual_train_size:actual_train_size + actual_val_size]
    test_df = df[actual_train_size + actual_val_size:actual_train_size + actual_val_size + actual_test_size]
    
    print(f"📊 최종 데이터 분할:")
    print(f"  - 학습: {len(train_df):,}개")
    print(f"  - 검증: {len(val_df):,}개") 
    print(f"  - 테스트: {len(test_df):,}개")
    
    return DatasetDict({
        "train": Dataset.from_pandas(train_df),
        "validation": Dataset.from_pandas(val_df),
        "test": Dataset.from_pandas(test_df)
    })

def load_audio_features(audio_path: str) -> np.ndarray:
    """기존 오디오 로딩 함수 그대로"""
    try:
        if audio_path.endswith('.npy'):
            features = np.load(audio_path)
        elif audio_path.endswith('.npz'):
            data = np.load(audio_path)
            key = list(data.keys())[0]
            features = data[key]
        else:
            raise ValueError(f"지원하지 않는 파일 형식: {audio_path}")
        
        if features.ndim == 1:
            features = features.reshape(-1, 1)
        elif features.ndim == 2:
            pass
        else:
            print(f"⚠️ 예상치 못한 특성 형태: {features.shape} in {audio_path}")
        
        return features
        
    except Exception as e:
        print(f"오디오 특성 로드 실패: {audio_path}, 오류: {e}")
        return np.zeros((1, 80))

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    """기존 데이터 콜레이터 그대로"""
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        labels = labels_batch["input_ids"].masked_fill(
            labels_batch.attention_mask.ne(1), -100
        )

        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels
        return batch

def prepare_dataset(batch, processor, feature_extractor):
    """기존 데이터 준비 함수 그대로"""
    audio_features = load_audio_features(batch["audio_path"])
    
    if audio_features.ndim == 2:
        if audio_features.shape[1] > audio_features.shape[0]:
            pass
        else:
            audio_features = audio_features.T
    
    target_time_steps = 3000  # Whisper 필수 요구사항
    
    if audio_features.shape[1] > target_time_steps:
        audio_features = audio_features[:, :target_time_steps]
    elif audio_features.shape[1] < target_time_steps:
        pad_width = target_time_steps - audio_features.shape[1]
        audio_features = np.pad(audio_features, ((0, 0), (0, pad_width)), mode='constant', constant_values=0)
    
    if audio_features.shape[0] != 80:
        if audio_features.shape[0] < 80:
            pad_features = 80 - audio_features.shape[0]
            audio_features = np.pad(audio_features, ((0, pad_features), (0, 0)), mode='constant', constant_values=0)
        else:
            audio_features = audio_features[:80, :]
    
    batch["input_features"] = audio_features.astype(np.float32)
    
    batch["labels"] = processor.tokenizer(
        batch["transcript"],
        truncation=True,
        max_length=config.MAX_LENGTH,
        padding=False,
        return_tensors="pt"
    ).input_ids[0]
    
    return batch

def compute_metrics(eval_pred, tokenizer):
    """메모리 절약한 메트릭 계산"""
    pred_ids, label_ids = eval_pred
    
    # 메모리 절약을 위해 샘플 수 제한
    if len(pred_ids) > config.MAX_EVAL_SAMPLES:
        pred_ids = pred_ids[:config.MAX_EVAL_SAMPLES]
        label_ids = label_ids[:config.MAX_EVAL_SAMPLES]
    
    label_ids[label_ids == -100] = tokenizer.pad_token_id
    
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    
    wer_metric = evaluate.load("wer")
    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    
    return {"wer": wer}

# =============================================================================
# 메인 학습 함수 - 체크포인트 없이 새로 시작
# =============================================================================

def train_from_scratch():
    """체크포인트 없이 처음부터 새로 시작하는 학습"""
    
    print("🚀 Whisper 한국어 파인튜닝 시작!")
    print("🆕 처음부터 새로 시작합니다!")
    print()
    print("🎯 학습 설정:")
    print(f"  - 모델: {config.MODEL_NAME}")
    print(f"  - 배치 크기: {config.BATCH_SIZE}")
    print(f"  - 그래디언트 누적: {config.GRADIENT_ACCUMULATION_STEPS}")
    print(f"  - 실효 배치: {config.BATCH_SIZE * config.GRADIENT_ACCUMULATION_STEPS}")
    print(f"  - 학습률: {config.LEARNING_RATE}")
    print(f"  - 에포크: {config.NUM_EPOCHS}")
    print(f"  - 체크포인트: {config.SAVE_STEPS}스텝마다")
    print()
    
    clear_memory()
    
    # 1. 데이터셋 로드
    print("📂 데이터셋 로딩...")
    dataset = load_dataset_from_directory(config.DATA_DIR)
    
    # 2. 모델 초기화 (항상 새로 시작)
    print("🤖 새 모델 초기화...")
    
    feature_extractor = WhisperFeatureExtractor.from_pretrained(config.MODEL_NAME)
    tokenizer = WhisperTokenizer.from_pretrained(
        config.MODEL_NAME, 
        language=config.LANGUAGE,
        task=config.TASK
    )
    processor = WhisperProcessor.from_pretrained(
        config.MODEL_NAME,
        language=config.LANGUAGE,
        task=config.TASK
    )
    model = WhisperForConditionalGeneration.from_pretrained(config.MODEL_NAME)
    
    # 언어 설정
    model.config.forced_decoder_ids = None
    model.config.suppress_tokens = []
    model.generation_config.language = config.LANGUAGE
    model.generation_config.task = config.TASK
    
    print("✅ 모델 준비 완료")
    
    # 3. 데이터 전처리
    print("🔄 데이터 전처리 중...")
    clear_memory()
    
    dataset = dataset.map(
        lambda batch: prepare_dataset(batch, processor, feature_extractor),
        remove_columns=dataset["train"].column_names,
        num_proc=1,
        batch_size=50
    )
    
    # 4. 데이터 콜레이터
    data_collator = DataCollatorSpeechSeq2SeqWithPadding(
        processor=processor,
        decoder_start_token_id=model.config.decoder_start_token_id,
    )
    
    # 5. 학습 설정
    training_args = Seq2SeqTrainingArguments(
        output_dir=config.OUTPUT_DIR,
        
        # 배치 설정
        per_device_train_batch_size=config.BATCH_SIZE,
        gradient_accumulation_steps=config.GRADIENT_ACCUMULATION_STEPS,
        per_device_eval_batch_size=config.EVAL_BATCH_SIZE,
        
        # 하이퍼파라미터
        learning_rate=config.LEARNING_RATE,
        num_train_epochs=config.NUM_EPOCHS,
        warmup_steps=config.WARMUP_STEPS,
        
        # 평가 최소화 (메모리 절약)
        evaluation_strategy="no",
        
        # 체크포인트 저장
        save_strategy="steps",
        save_steps=config.SAVE_STEPS,
        save_total_limit=config.SAVE_TOTAL_LIMIT,
        logging_steps=100,
        
        # 기타 설정
        remove_unused_columns=True,
        label_names=["labels"],
        load_best_model_at_end=False,
        push_to_hub=False,
        
        # 하드웨어 최적화
        dataloader_num_workers=config.DATALOADER_NUM_WORKERS,
        dataloader_pin_memory=False,
        fp16=config.USE_FP16,
        gradient_checkpointing=True,
        
        report_to=[],  # 텐서보드 끔
    )
    
    # 6. 트레이너 설정
    trainer = Seq2SeqTrainer(
        args=training_args,
        model=model,
        train_dataset=dataset["train"],
        data_collator=data_collator,
        tokenizer=processor.feature_extractor,
    )
    
    # 7. 학습 시작
    print("🎓 학습 시작!")
    print(f"📊 최종 설정 확인:")
    print(f"  - 학습 데이터: {len(dataset['train']):,}개")
    print(f"  - 총 스텝: 약 {len(dataset['train']) // (config.BATCH_SIZE * config.GRADIENT_ACCUMULATION_STEPS) * config.NUM_EPOCHS:,}")
    print(f"  - 예상 시간: 약 {(len(dataset['train']) // (config.BATCH_SIZE * config.GRADIENT_ACCUMULATION_STEPS) * config.NUM_EPOCHS) // 60}시간")
    print()
    
    clear_memory()
    
    try:
        # 처음부터 새로 시작
        print("🆕 처음부터 새로 학습 시작...")
        trainer.train()
        
        print("💾 최종 모델 저장 중...")
        trainer.save_model()
        processor.save_pretrained(config.OUTPUT_DIR)
        tokenizer.save_pretrained(config.OUTPUT_DIR)
        feature_extractor.save_pretrained(config.OUTPUT_DIR)
        
        # 최종 테스트 평가
        print("📊 최종 테스트 평가...")
        test_trainer = Seq2SeqTrainer(
            args=Seq2SeqTrainingArguments(
                output_dir=config.OUTPUT_DIR,
                per_device_eval_batch_size=1,
                fp16=True,
            ),
            model=model,
            data_collator=data_collator,
            compute_metrics=lambda eval_pred: compute_metrics(eval_pred, tokenizer),
            tokenizer=processor.feature_extractor,
        )
        
        test_results = test_trainer.evaluate(dataset["test"])
        print(f"🎯 최종 테스트 WER: {test_results['eval_wer']:.4f}")
        
        print("🎉 학습 완전 완료!")
        print(f"📁 모델 저장 위치: {config.OUTPUT_DIR}")
        
    except Exception as e:
        print(f"❌ 학습 중 오류 발생: {e}")
        print("💡 메모리 부족 시 추가 최적화 제안:")
        print("  - BATCH_SIZE = 2")
        print("  - GRADIENT_ACCUMULATION_STEPS = 16")
        print("  - TRAIN_SIZE = 20000")
        raise e

if __name__ == "__main__":
    train_from_scratch()

🚀 Whisper 한국어 파인튜닝 시작!
🆕 처음부터 새로 시작합니다!

🎯 학습 설정:
  - 모델: openai/whisper-small
  - 배치 크기: 4
  - 그래디언트 누적: 8
  - 실효 배치: 32
  - 학습률: 1e-05
  - 에포크: 3
  - 체크포인트: 1000스텝마다

📂 데이터셋 로딩...
📂 데이터셋 로딩 중...
📝 텍스트 파일 수: 124,000개
📊 진행 상황: 10,000개 매칭됨
📊 진행 상황: 20,000개 매칭됨
📊 진행 상황: 30,000개 매칭됨
📊 진행 상황: 40,000개 매칭됨
📊 진행 상황: 50,000개 매칭됨
📊 진행 상황: 60,000개 매칭됨
📊 진행 상황: 70,000개 매칭됨
📊 진행 상황: 80,000개 매칭됨
📊 진행 상황: 90,000개 매칭됨
📊 진행 상황: 100,000개 매칭됨
📊 진행 상황: 110,000개 매칭됨
📊 진행 상황: 120,000개 매칭됨
✅ 총 124,000개 파일 쌍 매칭됨
  - 유효한 데이터: 124,000개
📊 최종 데이터 분할:
  - 학습: 32,000개
  - 검증: 4,000개
  - 테스트: 4,000개
🤖 새 모델 초기화...
✅ 모델 준비 완료
🔄 데이터 전처리 중...


Map: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 4000/4000 [00:15<00:00, 263.28 examples/s]


🎓 학습 시작!
📊 최종 설정 확인:
  - 학습 데이터: 32,000개
  - 총 스텝: 약 3,000
  - 예상 시간: 약 50시간

🆕 처음부터 새로 학습 시작...


`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss
100,5.396500
200,4.017200
300,3.489800
400,3.288100
500,3.137600
600,3.082600
700,3.011200
800,2.963600
900,2.928200
1000,2.895900


💾 최종 모델 저장 중...
📊 최종 테스트 평가...


❌ 학습 중 오류 발생: CUDA out of memory. Tried to allocate 186.00 MiB. GPU 0 has a total capacity of 23.69 GiB of which 107.00 MiB is free. Process 264299 has 17.13 GiB memory in use. Process 2019775 has 6.43 GiB memory in use. Of the allocated memory 5.38 GiB is allocated by PyTorch, and 746.12 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
💡 메모리 부족 시 추가 최적화 제안:
  - BATCH_SIZE = 2
  - GRADIENT_ACCUMULATION_STEPS = 16
  - TRAIN_SIZE = 20000


OutOfMemoryError: CUDA out of memory. Tried to allocate 186.00 MiB. GPU 0 has a total capacity of 23.69 GiB of which 107.00 MiB is free. Process 264299 has 17.13 GiB memory in use. Process 2019775 has 6.43 GiB memory in use. Of the allocated memory 5.38 GiB is allocated by PyTorch, and 746.12 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)